In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    # at = Nt/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    # at = Nt/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

# without symmetry
## loading data

In [282]:
'''
without symmetry eigenvalue and eigenvector projection 
'''
# strfile='2021-03-27-16_16_38_perturbationHomoZeroeta_data' # 500
# strfile='2021-03-27-17_02_42_perturbationHomoZeroeta_data' # 1000
# strfile='2021-03-27-17_58_42_perturbationHomoZeroeta_data' # 750
# strfile='2021-03-27-20_31_10_perturbationHomoZeroeta_data' # 750
# strfile='2021-03-27-20_44_33_perturbationHomoZeroeta_data' # 750

# strfile='2021-03-27-20_01_56_perturbationHeterZeroeta_data' # 750
'''
previous data
'''
# strfile='2021-03-17-21_57_03_perturbationHomoZeroeta_data' # N=500
# strfile='2021-03-17-22_24_20_perturbationHomoZeroeta_data' #1000
# strfile='2021-03-17-20_35_21_perturbationHomoZeroeta_data' # 1000
# strfile='2021-03-17-22_46_39_perturbationHomoZeroeta_data'# 750
# '''# HETERO ZERO '''
# strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' 
data = np.load(strfile+'.npz')
gaverageseries=data['gaverageseries']
ngavg=len(gaverageseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
Xprimebasexee=data['Xprimebasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
x0,y0=data['x0'],data['y0']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)

scale: 750.0
JEE: 1.0
eta: [0. 0. 0.] ; signs: [1. 1. 1.]


In [241]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''

'''
# loading data
# strfile='2021-03-13-15_20_04_perturbationHomoZeroeta_data'# homogeneous random gain
strfile='2021-03-13-15_10_37_perturbationHeterZeroeta_data' # heterogeneous random gain
data = np.load(strfile+'.npz')
gaverageseries = data['gaverageseries']
x0,y0=data['x0'],data['y0']
meigvecAm,neigvecAm=data['meigvecAm'],data['neigvecAm']
# perturbed
Leigvecseries,Reigvecseries=data['Leigvecseries'],data['Reigvecseries']
Xprimebasexee=data['Xprimebasexee']
eigvAm=data['eigvAm']
ngavg,ntrial,N=np.shape(Leigvecseries)
Leigvecseries=np.expand_dims(Leigvecseries,axis=3)
Reigvecseries=np.expand_dims(Reigvecseries,axis=3)
NE=int(N/2.0)
'''

# '''
# calculate S0
# '''
# xticks = np.linspace(-0.08,0.08,2)
# xlims = [-0.08,0.08]
# yticks = np.linspace(-0.08,0.08,2)
# ylims = [-0.08,0.08]
# Z = np.zeros((NE*2-1,NE*2-1))
# for i in range(NE*2-1):
#     b1=eigvAm[i+1]
#     Z[i,i]=1/(-eigvAm[0])
# S0=np.zeros((NE*2,NE*2))
# R1,L1=np.squeeze(meigvecAm[:,1:]),np.squeeze(neigvecAm[:,1:])
# S0=R1@Z@L1.T

'''
# CHOOSE ONE TRIAL
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,3,figsize=(6,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:,i]),rperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:,i]),lperturb_num[:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(3):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
for i in range(3):
    ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

<ipython-input-241-cdfb9a45373f>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-241-cdfb9a45373f>:54: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)
<ipython-input-241-cdfb9a45373f>:56: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_the[:,i] = lprime_the*gsamples[i]
<ipython-input-241-cdfb9a45373f>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_the[:,i] = rprime_the*gsamples[i]
<ipython-input-241-cdfb9a45373f>:53: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-241-cdfb9a45373f>:54: ComplexWarning: Casting complex values to r

In [242]:
'''
### >>>>>>>>>>>>>>>>> Statistics Linearly Change >>>>>>>>>>>>>>>>
Case 1,2 focus on second-order statstics WITHIN (left/right) eigenvectors
Calculate the statistics of eigenvalue, eigenvectors
  * mean, variance of the perturbation of eigenvalue \mu(derivative of lambda),\sigma(derivative of lambda)
'''
'''
Variables:
    std_perturb_Lambda_num (1)
    std_perturb_Lambda_theo (1)
    std_perturbLambda_linear: reconstruct np.zeros(ngavg)
    std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
'''
alphanum=1.0
# figE,axE=plt.subplots(figsize=(4,4))
# figL0,axL0=plt.subplots(2,1,figsize=(5,4))
# figR0,axR0=plt.subplots(2,1,figsize=(5,4))

## calculating the statistics of perturbation lambda
# Coding uses only one point [linear assumption]
std_perturb_Lambda_numSE  = np.std(eigvalseries[ngavg-1,:,0])
std_perturb_Lambda_theo = np.sqrt((JE**2*(xee**2+xei**2)+JI**2*(xie**2+xii**2))/(2*NE)/(JE-JI)**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLambda_linear_theo=std_perturb_Lambda_theo*gaverageseries
std_perturbLambda_linear_numSE=std_perturb_Lambda_numSE*gaverageseries
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Lambda_num = np.std(Beigvseries[:,:],axis=1)
axE.plot(gaverageseries,std_perturb_Lambda_num,color='darkgreen',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# polyfitting
p=np.polyfit(gaverageseries,std_perturb_Lambda_num,1)
std_perturbLambda_linear_num = np.zeros(ngavg)
std_perturbLambda_linear_num = p[0]*gaverageseries+p[1]
# axE.plot(gaverageseries,std_perturbLambda_linear,color=cm[2],linestyle='--',linewidth=1.5,alpha=0.5,label='1-order polynomial')
axE.plot(gaverageseries,std_perturbLambda_linear_theo,color='green',linewidth=1.5,label='analytical',alpha=alphanum)
print('std delta_lambda numerical:',p[0])
axE.set_title(r'$\sigma_{\lambda}$')

'''
plt and ax
'''
ylims=[-0.00,0.08]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.08,2)
xticks = np.linspace(0.0,1.0,2)
axE.set_xlim(xlims)
axE.set_xticks(xticks)
axE.set_ylim(ylims)
axE.set_yticks(yticks)
# ax.legend()

'''
right eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Reig_numSE,std_perturb_Reig_theo=np.zeros(2),np.zeros(2)
std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbReig_linear_theo,std_perturbReig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbReig_linear_theo[:,0]=std_perturb_Reig_theo[0]*gaverageseries
std_perturbReig_linear_theo[:,1]=std_perturb_Reig_theo[1]*gaverageseries
# std_perturbReig_linear_numSE[:,0]=std_perturb_Reig_numSE[0]*gaverageseries
# std_perturbReig_linear_numSE[:,1]=std_perturb_Reig_numSE[1]*gaverageseries

print('std of perturbation right eigenvector, num:',std_perturb_Reig_numSE,'; theo:',std_perturb_Reig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Reig_num,std_perturbReig_linear_num=np.zeros((ngavg,2),dtype=complex),np.zeros((ngavg,2),dtype=complex)
std_perturb_Reig_num[:,0],std_perturb_Reig_num[:,1] = np.mean(statesReig[:,:,0,1],axis=1),np.mean(statesReig[:,:,1,1],axis=1)


# >>>>>>>> numerical solution
axR0[0].plot(gaverageseries,std_perturb_Reig_num[:,0],color='darkred',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
axR0[1].plot(gaverageseries,std_perturb_Reig_num[:,1],color='darkblue',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# axR0[0].scatter(gaverageseries,std_perturbReig_linear_numSE[:,0],color='orange',label='numerical')
# axR0[1].scatter(gaverageseries,std_perturbReig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axR0[0].plot(gaverageseries,std_perturbReig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical',alpha=alphanum)
axR0[1].plot(gaverageseries,std_perturbReig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical',alpha=alphanum)
axR0[0].set_title(r'$\sigma_{r}^E$')
axR0[1].set_title(r'$\sigma_{r}^I$')
# >>>>>>>> numerical solution
for i in range(ntrial):
    axR0[0].scatter(gaverageseries,statesReig[:,i,0,1],s=5,c='darkred',marker='^',label='numerical',alpha=alphanum*0.5)
    axR0[1].scatter(gaverageseries,statesReig[:,i,1,1],s=5,c='darkblue',marker='^',label='numerical',alpha=alphanum*0.5)

ylims=[-0.00,0.020]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.020,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axR0[i].set_xlim(xlims)
    axR0[i].set_xticks([])
    axR0[i].set_ylim(ylims)
    axR0[i].set_yticks(yticks)
axR0[1].set_xticks(xticks)

'''
left eigenvector
'''
## calculating the statistics of perturbation lambda
# Coding uses only one point
std_perturb_Leig_numSE,std_perturb_Leig_theo=np.zeros(2),np.zeros(2)
std_perturb_Leig_numSE[0],std_perturb_Leig_numSE[1] = np.mean(statesLeig[ngavg-1,:,0,1]),np.mean(statesLeig[ngavg-1,:,1,1])
std_perturb_Leig_theo[0],std_perturb_Leig_theo[1] = np.sqrt(2*((JE*xee)**2+(JI*xie)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2),np.sqrt(2*((JE*xei)**2+(JI*xii)**2)/(2*NE)/(JE-JI)**2/eigvAm[0]**2)
## theoretical curve for standard deviation of the eigenvalue
std_perturbLeig_linear_theo,std_perturbLeig_linear_numSE=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturbLeig_linear_theo[:,0]=std_perturb_Leig_theo[0]*gaverageseries
std_perturbLeig_linear_theo[:,1]=std_perturb_Leig_theo[1]*gaverageseries
std_perturbLeig_linear_numSE[:,0]=std_perturb_Leig_numSE[0]*gaverageseries
std_perturbLeig_linear_numSE[:,1]=std_perturb_Leig_numSE[1]*gaverageseries

print('std of perturbation left eigenvector, num:',std_perturb_Leig_numSE,'; theo:',std_perturb_Leig_theo)
'''
Variance of the perturbations in eigenvalue lambda
\delta lambda =\Delta lambda = (1) unit change in g
'''
std_perturb_Leig_num,std_perturbLeig_linear_num=np.zeros((ngavg,2)),np.zeros((ngavg,2))
std_perturb_Leig_num[:,0],std_perturb_Leig_num[:,1] = np.mean(statesLeig[:,:,0,1],axis=1),np.mean(statesLeig[:,:,1,1],axis=1)

# >>>>>>>> numerical solution
axL0[0].plot(gaverageseries,std_perturb_Leig_num[:,0],color='darkred',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
axL0[1].plot(gaverageseries,std_perturb_Leig_num[:,1],color='darkblue',linewidth=1.5,linestyle='--',label='numerical',alpha=alphanum)
# axL0[0].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,0],color='orange',label='numerical')
# axL0[1].scatter(gaverageseries,std_perturbLeig_linear_numSE[:,1],color='cyan',label='numerical')
# >>>>>>> Theoretical solutions
axL0[0].plot(gaverageseries,std_perturbLeig_linear_theo[:,0],color='red',linewidth=1.5,label='analytical',alpha=alphanum)
axL0[1].plot(gaverageseries,std_perturbLeig_linear_theo[:,1],color='blue',linewidth=1.5,label='analytical',alpha=alphanum)
axL0[0].set_title(r'$\sigma_{l}^E$')
axL0[1].set_title(r'$\sigma_{l}^I$')
axL0[1].set_xticks(xticks)
# >>>>>>>> numerical solution
for i in range(ntrial):
    axL0[0].scatter(gaverageseries,statesLeig[:,i,0,1],s=5,c='darkred',marker='^',label='numerical',alpha=alphanum*0.5)
    axL0[1].scatter(gaverageseries,statesLeig[:,i,1,1],s=5,c='darkblue',marker='^',label='numerical',alpha=alphanum*0.5)

ylims=[-0.00,0.045]
xlims=[-0.0,1.1]
yticks = np.linspace(0.0,0.045,2)
xticks = np.linspace(0.0,1.0,2)
for i in range(2):
    axL0[i].set_xlim(xlims)
    axL0[i].set_xticks([])
    axL0[i].set_ylim(ylims)
    axL0[i].set_yticks(yticks)
axL0[1].set_xticks(xticks)





std delta_lambda numerical: 0.041163186069500146
std of perturbation right eigenvector, num: [0.00966569 0.00973817] ; theo: [0.00894427 0.00894427]
std of perturbation left eigenvector, num: [0.01994889 0.01988402] ; theo: [0.01841738 0.01841738]
<ipython-input-242-169649c8e34f>:60: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_numSE[0],std_perturb_Reig_numSE[1] = np.mean(statesReig[ngavg-1,:,0,1]),np.mean(statesReig[ngavg-1,:,1,1])
<ipython-input-242-169649c8e34f>:61: ComplexWarning: Casting complex values to real discards the imaginary part
  std_perturb_Reig_theo[0],std_perturb_Reig_theo[1] = np.sqrt(((xee)**2+(xei)**2)/(2*2*NE)/eigvAm[0]**2),np.sqrt(((xie)**2+(xii)**2)/(2*2*NE)/eigvAm[0]**2)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/

In [58]:
from scipy.special import comb, perm
C=comb(4,2)
print(C)

6.0


## eigenvalue spectrum

In [ ]:
'''
first, generate eigenvalues' spectrum (we didn't record the whole eigenvalues' spectrum)
'''

In [206]:
'''
without symmetry, 
eigvAm
'''
idxgavg,idxtrial=0,8
figtspt,axtspt=plt.subplots(figsize=(8,5))
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=10,c='blue',alpha=0.5)
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0]),s=20,c='red',marker='^',alpha=0.5)
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
#设置x坐标刻度数字或名称的位置
axtspt.xaxis.set_ticks_position('bottom')
#设置边框位置
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverageseries[idxgavg]
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
eta=0
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="darkblue", linewidth=2.5,linestyle='--',label=r'ellipse')
avgg=np.sum(np.sum(gmat))*2/4.0
avgg=np.sqrt(avgg)
xa = avgg*np.cos(theta)
ya = avgg*np.sin(theta)
axtspt.plot(xa, ya, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse')
print(avgg,gaverage_)



1.0 1.0 1.0 1.0
0.1
0.1 0.1
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [243]:
'''
perturbation direction and amplitude
covariance - non-covariance
'''
'''
# <l',r'> reflects the direction of changing
'''

'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]
## reconstruct Am
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
'''
for reference
'''
x0,y0=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
gaverageseries=np.reshape(gaverageseries,(ngavg,1))
directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2))
ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2))
ntrial = np.shape(Leigvecseries)[1]
for itrial in range(ntrial):
    ## choose one trial and the random matrix
    # itrial=6
    derivJ=np.squeeze(Xprimebasexee[itrial,:,:])

    coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
    overlapperturb,overlapvector = np.zeros((neta,2,2,NE),dtype=complex),np.zeros((neta,2,2,NE),dtype=complex)
    # EI,LR,:
    Jprimemat=derivJ.copy()
    leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
    rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E

    # perturbEmat,perturbImat=np.zeros((2,NE)),np.zeros((2,NE))
    # perturbEmat[0,:],perturbEmat[1,:]=np.squeeze(leftperturb[:NE]),np.squeeze(rightperturb[:NE])
    # perturbImat[0,:],perturbImat[1,:]=np.squeeze(leftperturb[NE:]),np.squeeze(rightperturb[NE:])
    # pcovE,pcovI=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T
    # vpE,dirpE=la.eig(pcovE)
    # vpI,dirpI=la.eig(pcovI)
    # directionperturbE[i,itrial,:,:],directionperturbI[i,:,:]=dirpE,dirpI
    # ampperturbE[i,itrial,:],ampperturbI[i,itrial,:]=vpE*dh,vpI*dh

    overlapperturb[:,0,0,:],overlapperturb[:,0,1,:]=gaverageseries@np.reshape(leftperturb[:NE],(1,NE)),gaverageseries@np.reshape(rightperturb[:NE],(1,NE))
    overlapperturb[:,1,0,:],overlapperturb[:,1,1,:]=gaverageseries@np.reshape(leftperturb[NE:],(1,NE)),gaverageseries@np.reshape(rightperturb[NE:],(1,NE))
    meancol=np.reshape(np.mean(np.squeeze(Leigvecseries[:,itrial,:NE]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,0,0,:]=np.squeeze(Leigvecseries[:,itrial,:NE])-meancol
    meancol=np.reshape(np.mean(np.squeeze(Leigvecseries[:,itrial,NE:]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,1,0,:]=np.squeeze(Leigvecseries[:,itrial,NE:])-meancol
    meancol=np.reshape(np.mean(np.squeeze(Reigvecseries[:,itrial,:NE]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,0,1,:]=np.squeeze(Reigvecseries[:,itrial,:NE])-meancol
    meancol=np.reshape(np.mean(np.squeeze(Reigvecseries[:,itrial,NE:]),axis=1),(ngavg,1))
    meancol=np.repeat(meancol,NE,axis=1)
    overlapvector[:,1,1,:]=np.squeeze(Reigvecseries[:,itrial,NE:])-meancol


    for i in range(ngavg):
        vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
        perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
        vcovE,vcovI=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
        pcovE,pcovI=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

        vvE,dirvE=la.eig(vcovE)
        vvI,dirvI=la.eig(vcovI)
        vpE,dirpE=la.eig(pcovE)
        vpI,dirpI=la.eig(pcovI)
        directionvecE[i,itrial,:,:],directionvecI[i,:,:]=dirvE,dirvI
        directionperturbE[i,itrial,:,:],directionperturbI[i,:,:]=dirpE,dirpI
        ampvecE[i,itrial,:],ampvecI[i,itrial,:]=vvE*dh,vvI
        ampperturbE[i,itrial,:],ampperturbI[i,itrial,:]=vpE*dh,vpI*dh


## averaging
directionvecEavg,directionvecIavg,directionperturbEavg,directionperturbIavg=np.mean(directionvecE,axis=1),np.mean(directionvecI,axis=1),np.mean(directionperturbE,axis=1),np.mean(directionperturbI,axis=1)
ampvecEavg,ampvecIavg,ampperturbEavg,ampperturbIavg=np.mean(ampvecE,axis=1),np.mean(ampvecI,axis=1),np.mean(ampperturbE,axis=1),np.mean(ampperturbI,axis=1)



<ipython-input-243-9f8990a19c96>:46: RuntimeWarning: divide by zero encountered in true_divide
  coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
<ipython-input-243-9f8990a19c96>:88: ComplexWarning: Casting complex values to real discards the imaginary part
  directionvecE[i,itrial,:,:],directionvecI[i,:,:]=dirvE,dirvI
<ipython-input-243-9f8990a19c96>:89: ComplexWarning: Casting complex values to real discards the imaginary part
  directionperturbE[i,itrial,:,:],directionperturbI[i,:,:]=dirpE,dirpI
<ipython-input-243-9f8990a19c96>:90: ComplexWarning: Casting complex values to real discards the imaginary part
  ampvecE[i,itrial,:],ampvecI[i,itrial,:]=vvE*dh,vvI
<ipython-input-243-9f8990a19c96>:91: ComplexWarning: Casting complex values to real discards the imaginary part
  ampperturbE[i,itrial,:],ampperturbI[i,itrial,:]=vpE*dh,vpI*dh


In [246]:
# ## averaging
# directionvecEavg,directionvecIavg,directionperturbEavg,directionperturbIavg=np.mean(directionvecE,axis=1),np.mean(directionvecI,axis=1),np.mean(directionperturbE,axis=1),np.mean(directionperturbI,axis=1)
# ampvecEavg,ampvecIavg,ampperturbEavg,ampperturbIavg=np.mean(ampvecE,axis=1),np.mean(ampvecI,axis=1),np.mean(ampperturbE,axis=1),np.mean(ampperturbI,axis=1)

fig,ax=plt.subplots(figsize=(5,5)) 
for idxeta in np.arange(1,8):
    heta=gaverageseries[-1]*0.1
    # ax.plot([0,0+ampvecEavg[idxeta,0]*directionvecEavg[idxeta,0,0]],[0,0+ampvecEavg[idxeta,0]*directionvecEavg[idxeta,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecIavg[idxeta,0]*directionvecIavg[idxeta,0,0]],[0,0+ampvecIavg[idxeta,0]*directionvecIavg[idxeta,1,0]],color='orange',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)

    vecEx,vecEy=ampvecEavg[idxeta,0]*directionvecEavg[idxeta,0,0],ampvecEavg[idxeta,0]*directionvecEavg[idxeta,1,0]
    vecIx,vecIy=ampvecIavg[idxeta,0]*directionvecIavg[idxeta,0,0],ampvecIavg[idxeta,0]*directionvecIavg[idxeta,1,0]
    if idxeta<7:
        ax.plot([vecEx,vecEx+ampperturbEavg[idxeta,0]*directionperturbEavg[idxeta,0,0]*heta],[vecEy,vecEy+ampperturbEavg[idxeta,0]*directionperturbEavg[idxeta,1,0]*heta],color='b',linestyle='--',linewidth=1.5)
        ax.plot([vecIx,vecIx+ampperturbIavg[idxeta,0]*directionperturbIavg[idxeta,0,0]*heta],[vecIy,vecIy+ampperturbIavg[idxeta,0]*directionperturbIavg[idxeta,1,0]*heta],color='g',linestyle='--',linewidth=1.5)
    # ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,0]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,0]*heta],color='b',linestyle='--',linewidth=1.5)
    # ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,0]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,0]*heta],color='g',linestyle='--',linewidth=1.5)

xticks = np.linspace(-0.05,0.05,2)
xlims = [-0.05,0.05]
yticks = np.linspace(-0.05,0.05,2)
ylims = [-0.05,0.05]
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')

In [103]:
'''
loading patterns
'''
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.05,0.2,2)
# ylims = [-0.05,0.2]

## withoueigenvalue
xticks = np.linspace(-0.1,0.1,2)
xlims = [-0.1,0.1]
yticks = np.linspace(-0.02,0.08,2)
ylims = [-0.02,0.08]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
# pearsoncorr_rho=np.zeros((3,ngavg,neta,ntrial,2,nrank))
# thetamaxseries=np.zeros((3,ngavg,neta,ntrial,2,2))
# eigtotalseries=np.zeros((3,ngavg,neta,ntrial,nrank))
idxtrial=5#16#
idxgavg=4#9# 
cm='mc'
# fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# ## directly use
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0])*Beigvseries[idxgavg,idxtrial,0],np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])*Beigvseries[idxgavg,idxtrial,0]
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
# ## directly use without eigvalue
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
## reload from recording
mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:]),np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:])
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

## keep the same sign
if meanmE[0]<0:
    meanmE,meanmI,meannE,meannI=-meanmE,-meanmI,-meannE,-meannI
    mEvec,mIvec,nEvec,nIvec=-mEvec,-mIvec,-nEvec,-nIvec
''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=10.0,c=cm[1],alpha=0.25)
ax.scatter(nEvec,mEvec,s=10.0,c=cm[0],alpha=0.25)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=cm[1],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=cm[1],linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=cm[0])
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=cm[1])
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
print(gaverageseries[idxgavg])

            


750
(2, 2)
(2, 2)
0.5
<ipython-input-103-321248a7bc9d>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-103-321248a7bc9d>:49: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-103-321248a7bc9d>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-103-321248a7bc9d>:64: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-103-321248a7bc9d>:75: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-103-321248a7bc9d>:76: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


## INDIVIDUAL STATISTICS

In [283]:
'''
>>>>>>>>>>>>>>>>> Compare the individual response (not statistics), NOTE:
To make direct comparison between numerical and perturbation theorem, keep the same random matrix for each trial
choose one trial -- determine the random derivative -- calculate the individual elements in vector

'''
xticks = np.linspace(-0.1,0.1,2)
xlims = [-0.1,0.1]
yticks = np.linspace(-0.1,0.1,2)
ylims = [-0.1,0.1]
'''
# CHOOSE ONE TRIAL
'''
shiftepsilon=0.02
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,2,figsize=(4,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,9])#[1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:NE,i]),rperturb_num[:NE,i],s=5,c='black',alpha=0.25)
    ax[0,i].scatter(np.real(rperturb_the[NE:,i])+shiftepsilon,rperturb_num[NE:,i]-shiftepsilon,s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:NE,i]),lperturb_num[:NE,i],s=5,c='black',alpha=0.25)
    ax[1,i].scatter(np.real(lperturb_the[NE:,i])+shiftepsilon,lperturb_num[NE:,i]-shiftepsilon,s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])

for i in range(len(idxgsamples)):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
# for i in range(3):
#     ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])
'''
# CHOOSE TWO TRIALS
'''
idxtrial,idxtrial_=8,3
fig,ax=plt.subplots(2,2,figsize=(4,4))
Xprime = np.squeeze(Xprimebasexee[idxtrial,:,:])
idxgsamples=np.array([1,9])#[1,5,9])
gsamples=gaverageseries[idxgsamples]
rperturb_num,lperturb_num = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rperturb_the,lperturb_the = np.zeros((NE*2,3)),np.zeros((NE*2,3))
rprime_the,lprime_the = np.squeeze(Xprime@x0/eigvAm[0]),np.squeeze(Xprime.T@y0/eigvAm[0])
for i in range(len(idxgsamples)):
    lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(y0)
    rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial_,:])-np.squeeze(x0)

    lperturb_the[:,i] = lprime_the*gsamples[i]
    rperturb_the[:,i] = rprime_the*gsamples[i]
    ax[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    ax[0,i].scatter(np.real(rperturb_the[:NE,i]),rperturb_num[:NE,i],s=5,c='black',alpha=0.25)
    ax[0,i].scatter(np.real(rperturb_the[NE:,i]),rperturb_num[NE:,i],s=5,c='gray',alpha=0.5)
    ax[1,i].scatter(np.real(lperturb_the[:NE,i]),lperturb_num[:NE,i],s=5,c='black',alpha=0.25)
    ax[1,i].scatter(np.real(lperturb_the[NE:,i]),lperturb_num[NE:,i],s=5,c='gray',alpha=0.5)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])



for i in range(len(idxgsamples)):
    ax[0,i].set_xlim(xlims)
    ax[0,i].set_ylim(ylims)
    ax[1,i].set_xlim(xlims)
    ax[1,i].set_ylim(ylims)
ax[0,0].set_yticks(yticks)
ax[1,0].set_yticks(yticks)
# for i in range(2):
#     ax[1,i].set_xticks(xticks)
# print(np.shape(rprime_the),np.shape(rprime_num))
# ax.scatter(np.real(rprime_the),rprime_num,c='gray',alpha=0.5)
# ax.set_xlim([-0.05,0.05])
# ax.set_ylim([-0.05,0.05])

<ipython-input-283-2a10abe36eda>:24: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-283-2a10abe36eda>:25: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_num[:,i] = np.squeeze(Reigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(x0)
<ipython-input-283-2a10abe36eda>:27: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_the[:,i] = lprime_the*gsamples[i]
<ipython-input-283-2a10abe36eda>:28: ComplexWarning: Casting complex values to real discards the imaginary part
  rperturb_the[:,i] = rprime_the*gsamples[i]
<ipython-input-283-2a10abe36eda>:24: ComplexWarning: Casting complex values to real discards the imaginary part
  lperturb_num[:,i] = np.squeeze(Leigvecseries[idxgsamples[i],idxtrial,:])-np.squeeze(y0)
<ipython-input-283-2a10abe36eda>:25: ComplexWarning: Casting complex values to r

# with symmetry
## loading data

In [33]:
# strfile='2021-03-27-22_40_39_perturbationHomoeta_data' # 750 0.8
# strfile='2021-03-27-23_15_34_perturbationHomoeta_data' # 500 0.8
# strfile='2021-03-28-16_34_16_perturbationHomoeta_data' #1000 0.2

# strfile='2021-03-28-10_09_24_perturbationHomoeta_data' # 750 0.2
# strfile='2021-03-28-10_14_57_perturbationHomoeta_data' # 500 0.2
strfile='2021-03-28-16_15_47_perturbationHomoeta_data' # 1000 0.8

## reverse JE and JI
# strfile='2021-03-30-16_38_37_perturbationHomoeta_data' # 1000
# strfile='2021-03-30-16_55_54_perturbationHomoeta_data' # 750

## JI 4.5 JE 0.5
# strfile='2021-03-30-17_04_25_perturbationHomoeta_data' # 750
# strfile='2021-03-30-17_47_12_perturbationHomoeta_data' # 1000

## compare convert
# strfile='2021-03-31-12_40_37_perturbationHomoetaComp_data'
# strfile='2021-03-31-15_46_58_perturbationHomoetaComp_data'

## convert
# strfile='2021-03-28-23_26_33_perturbationConvert_data'
data = np.load(strfile+'.npz')
etaseries=data['etaseries']
neta=len(etaseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)
print('gain average:',gaverage)


scale: 1000.0
JEE: 1.0
eta: [1. 1. 1.] ; signs: [1. 1. 1.]
gain average: 0.8


In [14]:
'''
determinant lemma, odd terms 
even order terms should be maitained
combination
concrete version,
given cut-off order
'''
from scipy.special import comb, perm
gaverage
xticks = np.linspace(-2.81,-2.45,2)
xlims = [-2.81,-2.45]
yticks = np.linspace(-2.81,-2.45,2)
ylims = [-2.81,-2.45]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:]),axis=1)
cutoff = np.arange(5)

lambda_theo=np.zeros((len(cutoff),neta))

for idxeta,eta in enumerate(etaseries):
    Bcoeff=[1,-eigvAm[0]]
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    for idxo in range(len(cutoff)):       
        # cut off = 2
        lnum = idxo+1
        C=comb(2*lnum,lnum)
        B=np.squeeze(C*(nAm.T@la.matrix_power(bXXT,lnum)@mAm)*gaverage**(lnum*2))
        Bcoeff.append(0)
        Bcoeff.append(-B)
        # print(Bcoeff)
        lambda_test=np.roots(Bcoeff)
        lambda_theo[idxo,idxeta]=lambda_test[0]
figd,axd=plt.subplots(figsize=(5,5))  
axd.plot(lambda_num,lambda_theo[0,:],'b',label='2 cut off')
axd.plot(lambda_num,lambda_theo[1,:],'r',label='4 cut off')
axd.plot(lambda_num,lambda_theo[2,:],'m',label='6 cut off')
axd.plot(lambda_num,lambda_theo[3,:],'g',label='8 cut off')
axd.plot(lambda_num,lambda_num,'k',label='real')
axd.plot([-3.0,-2.4],[-3.0,-2.4],linestyle='--',color='gray')
axd.set_xlim(xlims)
axd.set_ylim(ylims)
axd.set_xticks(xticks)
axd.set_yticks(yticks)
axd.set_aspect('equal')
axd.legend()
'''
Error
'''
lambda_numm= np.repeat(np.reshape(lambda_num,(1,neta)),len(cutoff),axis=0)
Errorlambda = np.sum((lambda_theo-lambda_numm)**2,axis=1)
fige,axe=plt.subplots(figsize=(5,2))  
axe.plot(cutoff,np.squeeze(Errorlambda))



<ipython-input-14-e924b8f8f374>:46: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo[idxo,idxeta]=lambda_test[0]
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=or

NameError: name 'axe' is not defined

In [22]:
### >>>>>>>> eigenvalues v.s. eta (degree of symmetry)
fig,ax=plt.subplots(figsize=(4,2))  
ax.plot(etaseries,lambda_num,color='black',linestyle='-.',linewidth=1.5)
ax.plot(etaseries,lambda_theo[0,:],color='blue',linestyle='-',linewidth=1.5)
ax.plot(etaseries,lambda_theo[1,:],color='red',linestyle='-',linewidth=1.5)
ax.plot(np.linspace(-1.5,1.5,2),eigvAm[0]*np.ones(2),color='grey',linestyle='--',linewidth=1.5)
xticks = np.linspace(-0.1,1.1,2)
xlims = [-0.01,1.01]
yticks = np.linspace(-2.81,-2.45,2)
ylims = [-2.81,-2.45]

ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
# ax.set_aspect('equal')

/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [12]:
'''
determinant lemma, odd terms 
even order terms should be maitained
combination
'''
from scipy.special import comb, perm
gaverage
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    B2=np.squeeze(C*(nAm.T@la.matrix_power(bXXT,1)@mAm)*gaverage**2)
    coeff2 = [1,-eigvAm[0],0,-B2]
    print(coeff2)
    C=comb(4,2)
    B4=np.squeeze(C*(nAm.T@la.matrix_power(bXXT,2)@mAm)*gaverage**4)
    # B4=(nAm.T@(Y_t@YT_t@Y_t@YT_t)@mAm)*fory**2*foryt**2*6
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    print(coeff4)
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
ax.legend()

# fig,ax=plt.subplots(figsize=(5,3))  
# ax.plot(etaseries,lambda_num,'b',label='2 cut off')
# ax.set_aspect('equal')
# # ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# # ax.set_aspect('equal')


2.0
[1, (2.499999999999994-0j), 0, -0.0]
[1, (2.499999999999994-0j), 0, -0.0, 0, -0.0]
(-2.499999999999994+0j) (-2.499999999999994+0j)
2.0
[1, (2.499999999999994-0j), 0, 0.17777777777777767]
[1, (2.499999999999994-0j), 0, 0.17777777777777767, 0, 0.018962962962962952]
(-2.5278217585740457-5.551115123125783e-17j) (-2.5282758619206236+1.6653345369377348e-16j)
2.0
[1, (2.499999999999994-0j), 0, 0.35555555555555535]
[1, (2.499999999999994-0j), 0, 0.35555555555555535, 0, 0.07585185185185181]
(-2.554487863769375-1.1102230246251565e-16j) (-2.5561918545671674-2.220446049250313e-16j)
2.0
[1, (2.499999999999994-0j), 0, 0.533333333333333]
<ipython-input-12-a4b3757ab3e6>:44: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo2[idxeta]=lambda_test2[0]
<ipython-input-12-a4b3757ab3e6>:46: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo4[idxeta]=lambda_test4[0]
[1, (2.499999999999994-0j), 0, 0.533333333333333, 0, 0.1706666666

In [13]:
'''
determinant lemma, odd terms 
even order terms should be maitained
original Algorithm, miss terms
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
y0E,y0I=np.mean(y0[:NE,0]),np.mean(y0[NE:,0])
x0E,x0I=np.mean(x0[:NE,0]),np.mean(x0[NE:,0])
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:]),axis=1)
cutoff = 2
for idxeta,eta in enumerate(etaseries):
    etaf,etam,etal=eta*coeff[0],coeff[1]*eta,eta*coeff[2]
    bcoeffE0,bcoeffI0=(signeta[0]*etaf+signeta[1]*etam)*1/2.0,(signeta[2]*etal+signeta[1]*etam)*1/2.0
    print(bcoeffI0,bcoeffE0)
    B2=(bcoeffE0*JE-bcoeffI0*JI)*gaverage**2
    coeff2 = [1,-eigvAm[0],0,-B2]
    B4=(bcoeffE0**2*JE-bcoeffI0**2*JI)*gaverage**4
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    # B2=(bcoeffE0*y0E*x0E*NE)+(bcoeffI0*y0I*x0I*NE)
    # coeff2 = [1,-eigvJ0[0],0,-B2*eigvJ0[0]]
    # B4=(bcoeffE0**2*y0E*x0E*NE)+(bcoeffI0**2*y0I*x0I*NE)
    # coeff4 = [1,-eigvJ0[0],0,-B2*eigvJ0[0],0,-B4*eigvJ0[0]]#(eta*eigvAm[0]*gaverage])
    lambda_test2=np.roots(coeff2)
    lambda_theo2[idxeta]=lambda_test2[0]
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]
    print(lambda_test2[0],lambda_test4[0])
fig,ax=plt.subplots(figsize=(5,5))  
ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_theo,'b')
ax.plot(lambda_num,lambda_num,'k',label='real')
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
ax.legend()

fig,ax=plt.subplots(figsize=(5,3))  
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')
# ax.scatter(np.real(Beigvseries[neta-5,8,1:]),np.imag(Beigvseries[neta-5,8,1:]))
# ax.set_aspect('equal')



0.0 0.0
(-2.499999999999994+0j) (-2.499999999999994+0j)
0.1111111111111111 0.1111111111111111
(-2.5278217585740457-5.551115123125783e-17j) (-2.5281245660286054+2.220446049250313e-16j)
0.2222222222222222 0.2222222222222222
(-2.554487863769375-1.1102230246251565e-16j) (-2.55562485065333-1.6653345369377348e-16j)
0.3333333333333333 0.3333333333333333
(-2.5801161747594694+2.7755575615628914e-16j) (-2.5825246849964394-4.996003610813204e-16j)
0.4444444444444444 0.4444444444444444
(-2.604806123125268+1.1102230246251565e-16j) (-2.6088481722478356+2.220446049250313e-16j)
0.5555555555555556 0.5555555555555556
(-2.628642487993093+5.551115123125783e-16j) (-2.6346191862097452-1.1102230246251565e-16j)
0.6666666666666666 0.6666666666666666
(-2.651698233959279+1.1102230246251565e-16j) (-2.65986103923186+7.771561172376096e-16j)
0.7777777777777777 0.7777777777777777
(-2.674036681020125-6.661338147750939e-16j) (-2.684596269929113+7.771561172376096e-16j)
0.8888888888888888 0.8888888888888888
(-2.6957131889

In [47]:
'''
# <l',r'> reflects the direction of changing
'''

'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]
## reconstruct Am
nrank=1
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
vcovEmat,vcovImat=np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2))
pcovEmat,pcovImat=np.zeros((neta,ntrial,2,2)),np.zeros((neta,ntrial,2,2))
directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2))
ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2))
ntrial = np.shape(Leigvecseries)[1]
for itrial in range(ntrial):
    ## choose one trial and the random matrix
    # itrial=6
    gY=np.squeeze(gYbasexee[itrial,:,:])
    # # NORMAL
    # x0,y0=np.reshape(Reigvecseries[0,itrial,:],(2*NE,1)),np.reshape(Leigvecseries[0,itrial,:],(2*NE,1))
    # COMPARE CONVERT
    x0,y0=np.reshape(Reigvecseries[0,itrial,:],(2*NE,1)),np.reshape(Leigvecseries[0,itrial,:],(2*NE,1)) # 0 is after calculation
    
    # ## reconstruct the non-symmetric matrix (original random matrix with \eta=0)
    # X0=gY.copy()
    # J0=Am.copy()+X0.copy()
    # ''' # calculate the original \bar{m} and \bar{n} '''
    # eigvJ0,eigrvecJ0=la.eig(J0)
    # inveigrvecJ0=la.inv(eigrvecJ0)
    # meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    # neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    # neig0=neig0.copy().T
    # print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    # for i in range(nrank):
    #     neig0[:,i]*=eigvJ0[i]
    # for j in range(1,N):
    #     neig0[:,j]*=eigvJ0[j]

    # leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    # ''' The first '''
    # reig0=meig0.copy()
    # normval=np.sum(reig0*neig0.copy(),axis=0)
    # normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    # leig0=neig0.copy()/normval.copy()
    # ''' calculate s0'''
    # Z = np.zeros((NE*2-1,NE*2-1))
    # for i in range(NE*2-1):
    #     b1=eigvJ0[i+1]
    #     Z[i,i]=1/(b1-eigvJ0[0])
    # S0=reig0[:,1:]@Z@leig0[:,1:].T
    # print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    # print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    # print('eigenvalues:',eigvJ0[:nrank])
    # '''## for reference '''
    # x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    # ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    # tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    # ## make sure the E in x is positive, if negative then all x and y should change sign
    # if (np.mean(tildex[:NE])*xAm[0,0])<0:
    #     tildex*=(-1)
    #     tildey*=(-1)
    # x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    # x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    # y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    # x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
    

    '''
    rho_prime=eta/(2*sqrt(1-eta**2))
    rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
    rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
    '''
    dh=1#etaseries[1]-etaseries[0]
    rhoseries=(1-np.sqrt(1-etaseries**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
    sqrt_1meta2= np.sqrt(1-etaseries**2)
    coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
    overlapperturb,overlapvector = np.zeros((neta,2,2,NE),dtype=complex),np.zeros((neta,2,2,NE),dtype=complex)
    # EI,LR,:
    for i in range(neta-1):
        Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T-np.sqrt(coeffsqrtY[i])*gY
        leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
        rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E
        overlapperturb[i,0,0,:],overlapperturb[i,0,1,:]=leftperturb[:NE],rightperturb[:NE]
        overlapperturb[i,1,0,:],overlapperturb[i,1,1,:]=leftperturb[NE:],rightperturb[NE:]
        overlapvector[i,0,0,:]=np.squeeze(Leigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Leigvecseries[i,itrial,:NE]))
        overlapvector[i,1,0,:]=np.squeeze(Leigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Leigvecseries[i,itrial,NE:]))
        overlapvector[i,0,1,:]=np.squeeze(Reigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Reigvecseries[i,itrial,:NE]))
        overlapvector[i,1,1,:]=np.squeeze(Reigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Reigvecseries[i,itrial,NE:]))
    ## E population 
    # vectorcovE,perturbationcovE = np.zeros((neta,2,2)),np.zeros((neta,2,2))

    for i in range(neta):
        vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
        perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
        vcovEmat[i,itrial,:,:],vcovImat[i,itrial,:,:]=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
        pcovEmat[i,itrial,:,:],pcovImat[i,itrial,:,:]=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

## averaging
vcovEmatavg,vcovImatavg=np.mean(vcovEmat,axis=1),np.mean(vcovImat,axis=1)
pcovEmatavg,pcovImatavg=np.mean(pcovEmat,axis=1),np.mean(pcovImat,axis=1)
for i in range(neta):
    vvE,dirvE=la.eig(np.squeeze(vcovEmatavg[i,:,:]))
    vvI,dirvI=la.eig(np.squeeze(vcovImatavg[i,:,:]))
    vpE,dirpE=la.eig(np.squeeze(pcovEmatavg[i,:,:]))
    vpI,dirpI=la.eig(np.squeeze(pcovImatavg[i,:,:]))
    directionvecE[i,:,:],directionvecI[i,:,:]=dirvE,dirvI
    directionperturbE[i,:,:],directionperturbI[i,:,:]=dirpE,dirpI
    ampvecE[i,:],ampvecI[i,:]=vvE*dh,vvI*dh
    ampperturbE[i,:],ampperturbI[i,:]=vpE*dh,vpI*dh



<ipython-input-47-6c73e322592b>:86: RuntimeWarning: divide by zero encountered in true_divide
  coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
<ipython-input-47-6c73e322592b>:105: ComplexWarning: Casting complex values to real discards the imaginary part
  vcovEmat[i,itrial,:,:],vcovImat[i,itrial,:,:]=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
<ipython-input-47-6c73e322592b>:106: ComplexWarning: Casting complex values to real discards the imaginary part
  pcovEmat[i,itrial,:,:],pcovImat[i,itrial,:,:]=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T


In [48]:
# ## averaging
# directionvecEavg,directionvecIavg,directionperturbEavg,directionperturbIavg=np.mean(directionvecE,axis=1),np.mean(directionvecI,axis=1),np.mean(directionperturbE,axis=1),np.mean(directionperturbI,axis=1)
# ampvecEavg,ampvecIavg,ampperturbEavg,ampperturbIavg=np.mean(ampvecE,axis=1),np.mean(ampvecI,axis=1),np.mean(ampperturbE,axis=1),np.mean(ampperturbI,axis=1)

fig,ax=plt.subplots(figsize=(5,5)) 
for idxeta in np.arange(1,9):
    heta=1
    ax.plot([0,0+ampvecE[idxeta,0]*directionvecE[idxeta,0,0]],[0,0+ampvecE[idxeta,0]*directionvecE[idxeta,1,0]],color='r',linestyle='-',linewidth=1.5)
    ax.plot([0,0+ampvecI[idxeta,0]*directionvecI[idxeta,0,0]],[0,0+ampvecI[idxeta,0]*directionvecI[idxeta,1,0]],color='orange',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)
    if idxeta==4:
        vecEx,vecEy=ampvecE[idxeta,0]*directionvecE[idxeta,0,0],ampvecE[idxeta,0]*directionvecE[idxeta,1,0]
        vecIx,vecIy=ampvecI[idxeta,0]*directionvecI[idxeta,0,0],ampvecI[idxeta,0]*directionvecI[idxeta,1,0]
        if idxeta<8:
            ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,1]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,1]*heta],color='b',linestyle='--',linewidth=1.5)
            ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,1]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,1]*heta],color='g',linestyle='--',linewidth=1.5)
    # ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,0]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,0]*heta],color='b',linestyle='--',linewidth=1.5)
    # ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,0]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,0]*heta],color='g',linestyle='--',linewidth=1.5)

xticks = np.linspace(-0.15,0.15,2)
xlims = [-0.15,0.15]
yticks = np.linspace(-0.15,0.15,2)
ylims = [-0.15,0.15]
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')

In [35]:
# strfile='2021-03-27-22_40_39_perturbationHomoeta_data' # 750 0.8
# strfile='2021-03-28-16_15_47_perturbationHomoeta_data' # 1000 0.8

## JI 4.5 JE 0.5
# strfile='2021-03-30-17_04_25_perturbationHomoeta_data' # 750
strfile='2021-03-30-17_47_12_perturbationHomoeta_data' # 1000

data = np.load(strfile+'.npz')
etaseries=data['etaseries']
neta=len(etaseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
etaset=data['etaset']
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)
print('gain average:',gaverage)


DeltaLR_matheta,DeltaLR_mathg,DeltaLR_num = np.zeros((neta,4),dtype=complex),np.zeros((neta,4),dtype=complex),np.zeros((neta,ntrial,4),dtype=complex)
DeltaLR_eigv = np.zeros((neta,ntrial),dtype=complex)
Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
## initial vectors LEFT & RIGHT same for identical trial
ppercentEsub=0.5*np.ones(2)
coeffetaEsub = coeffeta[0]*np.ones(3)
coeffetaTotal = np.ones(3)
coeffetaTotal[0],coeffetaTotal[1],coeffetaTotal[2]=coeffeta[1]*coeffetaTotal[0],coeffeta[1]*coeffetaTotal[1],coeffeta[2]*coeffetaTotal[2]
signetaEsub = np.ones(3)
signetaTotal = np.ones(3)
numE1=int(ppercentEsub[0]*NE)
for itrial in range(ntrial):
    # initial eigenvectors
    Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[0,itrial,:]).copy(),np.squeeze(Reigvecseries[0,itrial,:]).copy()
    l0,r0=np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#np.reshape(Leigvecinit[itrial,:],(2*NE,1)),np.reshape(Reigvecinit[itrial,:],(2*NE,1))

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        eta = etaseries[idxeta]
        etaE1,etaE12B,etaE2=eta*coeffetaEsub[0],eta*coeffetaEsub[1],eta*coeffetaEsub[2]
        etaE1IB,etaE2IB,etaI=eta*coeffetaTotal[0],eta*coeffetaTotal[1],eta*coeffetaTotal[2]
        rhoE1,rhoE12B,rhoE2= (1-np.sqrt(1-etaE1**2))/2.0,(1-np.sqrt(1-etaE12B**2))/2.0,(1-np.sqrt(1-etaE2**2))/2.0
        rhoE1IB,rhoE2IB,rhoI= (1-np.sqrt(1-etaE1IB**2))/2.0,(1-np.sqrt(1-etaE2IB**2))/2.0,(1-np.sqrt(1-etaI**2))/2.0

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:].copy()),(NE*2,1))+l0
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:].copy()),(NE*2,1))+r0

        DeltaLR_num[idxeta,itrial,3]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:numE1,0]*changeR_num[:numE1,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[numE1:NE,0]*changeR_num[numE1:NE,0])
        DeltaLR_num[idxeta,itrial,2]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])

        DeltaLR_eigv[idxeta,itrial]= (Beigvseries[idxeta,itrial]-eigvAm[0])/eigvAm[0]

        '''
        ### >>>>>>>>>>> delta in eta
        '''       
        DeltaLR_matheta[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]**2)*(etaE1-2.0*np.sqrt(rhoE1))*signetaEsub[0]/4.0
        DeltaLR_matheta[idxeta,0]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*ppercentEsub[1])*(etaE12B-2.0*np.sqrt(rhoE12B))*signetaEsub[1]/4.0 
        DeltaLR_matheta[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*1.0)*(etaE1IB-2.0*np.sqrt(rhoE1IB))*signetaTotal[0]/4.0)
        # print('DeltaE1:',DeltaLR_matheta[idxeta,0])

        DeltaLR_matheta[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]**2)*(etaE2-2.0*np.sqrt(rhoE2))*signetaEsub[2]/4.0 
        DeltaLR_matheta[idxeta,1]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*ppercentEsub[0])*(etaE12B-2.0*np.sqrt(rhoE12B))*signetaEsub[1]/4.0 
        DeltaLR_matheta[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*1.0)*(etaE2IB-2.0*np.sqrt(rhoE2IB))*signetaTotal[1]/4.0)
        # print('DeltaE2:',DeltaLR_matheta[idxeta,1])

        DeltaLR_matheta[idxeta,2]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[0])*(etaE1IB-2.0*np.sqrt(rhoE1IB))*signetaTotal[0]/4.0 
        DeltaLR_matheta[idxeta,2]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[1])*(etaE2IB-2.0*np.sqrt(rhoE2IB))*signetaTotal[1]/4.0 
        DeltaLR_matheta[idxeta,2]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*1.0)*(etaI-2.0*np.sqrt(rhoI))*signetaTotal[2]/4.0)
        '''
        ### >>>>>>>>>>> delta in g with eta
        '''  
        DeltaLR_mathg[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]**2)*(np.sqrt(rhoE1))*signetaEsub[0]/2.0
        DeltaLR_mathg[idxeta,0]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*ppercentEsub[1])*(np.sqrt(rhoE12B))*signetaEsub[1]/2.0 
        DeltaLR_mathg[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[0]*1.0)*(np.sqrt(rhoE1IB))*signetaTotal[0]/2.0)
        # print('DeltaE1_g:',DeltaLR_mathg[idxeta,0])

        DeltaLR_mathg[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]**2)*(np.sqrt(rhoE2))*signetaEsub[2]/2.0 
        DeltaLR_mathg[idxeta,1]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*ppercentEsub[0])*(np.sqrt(rhoE12B))*signetaEsub[1]/2.0 
        DeltaLR_mathg[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercentEsub[1]*1.0)*(np.sqrt(rhoE2IB))*signetaTotal[1]/2.0)
        # print('DeltaE2:',DeltaLR_mathg[idxeta,1])

        DeltaLR_mathg[idxeta,2]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[0])*(np.sqrt(rhoE1IB))*signetaTotal[0]/2.0 
        DeltaLR_mathg[idxeta,2]+=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*ppercentEsub[1])*(np.sqrt(rhoE2IB))*signetaTotal[1]/2.0 
        DeltaLR_mathg[idxeta,2]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(1.0*1.0)*(np.sqrt(rhoI))*signetaTotal[2]/2.0)

        DeltaLR_matheta[idxeta,3]=2*DeltaLR_matheta[idxeta,2]+2*DeltaLR_matheta[idxeta,1]+2*DeltaLR_matheta[idxeta,0]
        DeltaLR_mathg[idxeta,3]=2*DeltaLR_mathg[idxeta,2]+2*DeltaLR_mathg[idxeta,1]+2*DeltaLR_mathg[idxeta,0]

# fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for itrial in range(ntrial):
    ax.plot(etaseries,np.squeeze(DeltaLR_num[:,itrial,3]),'^',color='gray',markersize=5,alpha=0.25)
    ax.plot(etaseries+0.01,np.squeeze(DeltaLR_eigv[:,itrial]),'o',color='red',markersize=5,alpha=0.25)
ax.plot(etaseries,DeltaLR_matheta[:,3]+DeltaLR_mathg[:,3],color=cl[2],linewidth=2.0)

xticks = np.linspace(-0.,1.,2)
xlims = [-0.1,1.1]
yticks = np.linspace(0.0,0.12,2)
ylims = [-0.02,0.13]
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
# ax.set_aspect('equal')

scale: 1000.0
JEE: 0.5
eta: [1. 1. 1.] ; signs: [1. 1. 1.]
gain average: 0.8
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core

In [34]:
'''
<l,r> OVERLAP -- MATHEMATICAL SOLUTION
'''
'''
Total Theoretical Solution for Covariance
'''

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
ppercent = np.ones(3)
DeltaL_theo,DeltaR_theo,DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_math,DeltaLR_theo,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial),dtype=complex),np.zeros((neta,ntrial,3),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
## initial vectors LEFT & RIGHT same for identical trial

for itrial in range(ntrial):
    # initial eigenvectors
    Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[0,itrial,:]).copy(),np.squeeze(Reigvecseries[0,itrial,:]).copy()
    l0,r0=np.reshape(Leigvecinit[itrial,:],(2*NE,1)),np.reshape(Reigvecinit[itrial,:],(2*NE,1))#np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        etaloop = etaseries[idxeta]*np.ones(3)
        # print('eta:',etaloop)
        etaloop=etaloop*coeffeta
        # etaloop=(etaloop*(ppercent**2)
        print('equivalent etaloop:',etaloop)

        asqr=(1-np.sqrt(1-etaloop[0]**2))/2.0 
        rhoE=asqr

        asqr=(1-np.sqrt(1-etaloop[1]**2))/2.0 
        rhoB=asqr

        asqr=(1-np.sqrt(1-etaloop[2]**2))/2.0 
        rhoI=asqr

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:]),(NE*2,1))-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:]),(NE*2,1))-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        DeltaL_num[idxeta,itrial,:],DeltaR_num[idxeta,itrial,:]=np.squeeze(changeL_num),np.squeeze(changeR_num)
        DeltaLR_num[idxeta,itrial,2]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])
   
        # EXCITATORY
        # JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE)    
        # -JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)    
        DeltaLR_math[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE))    
        DeltaLR_math[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)))
        # Inhibitory
        DeltaLR_math[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB))    
        DeltaLR_math[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[2]**2)*(np.sqrt((1-rhoI)*rhoI)-np.sqrt(rhoI)))

        DeltaLR_math[idxeta,2]=DeltaLR_math[idxeta,1]+DeltaLR_math[idxeta,0]


equivalent etaloop: [0. 0. 0.]
equivalent etaloop: [0.11111111 0.11111111 0.11111111]
equivalent etaloop: [0.22222222 0.22222222 0.22222222]
equivalent etaloop: [0.33333333 0.33333333 0.33333333]
equivalent etaloop: [0.44444444 0.44444444 0.44444444]
equivalent etaloop: [0.55555556 0.55555556 0.55555556]
equivalent etaloop: [0.66666667 0.66666667 0.66666667]
equivalent etaloop: [0.77777778 0.77777778 0.77777778]
equivalent etaloop: [0.88888889 0.88888889 0.88888889]
equivalent etaloop: [1. 1. 1.]
equivalent etaloop: [0. 0. 0.]
equivalent etaloop: [0.11111111 0.11111111 0.11111111]
equivalent etaloop: [0.22222222 0.22222222 0.22222222]
equivalent etaloop: [0.33333333 0.33333333 0.33333333]
equivalent etaloop: [0.44444444 0.44444444 0.44444444]
equivalent etaloop: [0.55555556 0.55555556 0.55555556]
equivalent etaloop: [0.66666667 0.66666667 0.66666667]
equivalent etaloop: [0.77777778 0.77777778 0.77777778]
equivalent etaloop: [0.88888889 0.88888889 0.88888889]
equivalent etaloop: [1. 1. 

In [45]:
xticks = np.linspace(0.0,1.0,2)
xlims = [-0.1,1.1]
yticks = np.linspace(-0.04,0.0,2)
ylims = [-0.05,0.01]
# fig,ax=plt.subplots(figsize=(5,5))
# cl='rbk'
# for i in range(3):
#     ax.plot(DeltaLR_math[:,i],np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,label='num v.s. theo')
# # ax.plot(np.mean(DeltaLR_theo,axis=1),np.mean(DeltaLR_num,axis=1),color='b',linewidth=1.5,label='num v.s. simp')
# ax.set_aspect('equal')

fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for i in range(3):
    ax.plot(etaseries,DeltaLR_math[:,i],color=cl[i],linewidth=1.5)
    # ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,linestyle='--')
ax.plot(etaseries,DeltaLR_math[:,0],color=cl[0],linewidth=1.5,linestyle='--',alpha=1)
# fig,ax=plt.subplots(figsize=(5,5))
# cl='rbk'
# for i in range(2,3):
#     ax.plot(etaseries,DeltaLR_math[:,i],color=cl[i],linewidth=1.5)
#     ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,linestyle='--')

ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)


/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [210]:
'''
loading patterns with symmetry
'''

## withoueigenvalue
xticks = np.linspace(-0.0,1.0,2)
xlims = [-0.1,1.1]
yticks = np.linspace(-0.02,0.08,2)
ylims = [-0.02,0.08]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
idxtrial=3#16#
idxgavg=1#9# 
cm='mc'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# ## directly use
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0])*Beigvseries[idxgavg,idxtrial,0],np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])*Beigvseries[idxgavg,idxtrial,0]
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
# ## directly use without eigvalue
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
## reload from recording
mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:]),np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:])
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE,2,nrank)),np.zeros((NE,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=10.0,c=cm[1],alpha=0.25)
ax.scatter(nEvec,mEvec,s=10.0,c=cm[0],alpha=0.25)

ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=cm[0],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=cm[1],linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=cm[1],linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=cm[0])
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=cm[1])
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')
print(gaverageseries[idxgavg])

            


500
(2, 2)
(2, 2)
0.2
<ipython-input-210-ac8a17544e29>:35: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-210-ac8a17544e29>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-210-ac8a17544e29>:50: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-210-ac8a17544e29>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-210-ac8a17544e29>:64: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-210-ac8a17544e29>:65: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


In [255]:
'''
loading patterns
for ensemble-averaging results
'''
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.05,0.2,2)
# ylims = [-0.05,0.2]

# ## g = 0.2
# xticks = np.linspace(-0.05,0.1,2)
# xlims = [-0.05,0.1]
# yticks = np.linspace(0.01,0.04,2)
# ylims = [0.01,0.04]

# ## g = 0.8
# xticks = np.linspace(-0.1,0.15,2)
# xlims = [-0.1,0.15]
# yticks = np.linspace(-0.05,0.02,2)
# ylims = [-0.05,0.02]
# ## g = 0.8
# xticks = np.linspace(-0.1,0.15,2)
# xlims = [-0.1,0.15]
# yticks = np.linspace(-0.01,0.06,2)
# ylims = [-0.01,0.06]

## compare
xticks = np.linspace(-0.15,0.20,2)
xlims = [-0.15,0.20]
yticks = np.linspace(-0.05,0.1,2)
ylims = [-0.05,0.1]

theta = np.linspace(0, 2 * np.pi, 200)

idxtrial=15#16#
idxeta=7#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# ## directly use
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0])*Beigvseries[idxgavg,idxtrial,0],np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])*Beigvseries[idxgavg,idxtrial,0]
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
# ## directly use without eigvalue
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
## reload from recording
mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE]),np.squeeze(Reigvecseries[idxeta,:,NE:]),np.squeeze(Leigvecseries[idxeta,:,:NE]),np.squeeze(Leigvecseries[idxeta,:,NE:])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    # meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    # varmE = mEvec - meanmEtotal
    # idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    # mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # # puring I
    # meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    # varmI = mIvec - meanmItotal
    # idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    # mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # # n vector
    # # puring E
    # meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    # varnE = nEvec - meannEtotal
    # idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    # nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # # puring I
    # meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    # varnI = nIvec - meannItotal
    # idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    # nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

# ## keep the same sign
# if meanmE[0]<0:
#     meanmE,meanmI,meannE,meannI=-meanmE,-meanmI,-meannE,-meannI
#     mEvec,mIvec,nEvec,nIvec=-mEvec,-mIvec,-nEvec,-nIvec
''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=5.0,c=cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c=cm[0],alpha=alphaval)

# ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='gray',linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
print(etaseries[idxeta])

1000
(2, 2)
(2, 2)
0.7777777777777777
<ipython-input-255-d63cbc86c3c8>:62: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-255-d63cbc86c3c8>:69: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-255-d63cbc86c3c8>:77: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-255-d63cbc86c3c8>:84: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-255-d63cbc86c3c8>:95: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-255-d63cbc86c3c8>:96: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


In [88]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.05,0.2,2)
# ylims = [-0.05,0.2]

# ## g = 0.2
# xticks = np.linspace(-0.05,0.1,2)
# xlims = [-0.05,0.1]
# yticks = np.linspace(0.01,0.04,2)
# ylims = [0.01,0.04]
## g = 0.8
xticks = np.linspace(-0.1,0.15,2)
xlims = [-0.1,0.15]
yticks = np.linspace(-0.1,0.0,2)
ylims = [-0.1,0.0]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
# pearsoncorr_rho=np.zeros((3,ngavg,neta,ntrial,2,nrank))
# thetamaxseries=np.zeros((3,ngavg,neta,ntrial,2,2))
# eigtotalseries=np.zeros((3,ngavg,neta,ntrial,nrank))

# # for specific condition
# idxtrial=5#16#
# idxgavg=4#9# 

idxtrial=15#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)
# ## directly use
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0])*Beigvseries[idxgavg,idxtrial,0],np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])*Beigvseries[idxgavg,idxtrial,0]
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
# ## directly use without eigvalue
# mEvec,mIvec=np.squeeze(Reigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Reigvecseries[idxgavg,idxtrial,NE:,0])
# nEvec,nIvec=np.squeeze(Leigvecseries[idxgavg,idxtrial,:NE,0]),np.squeeze(Leigvecseries[idxgavg,idxtrial,NE:,0])
## reload from recording
mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE])*np.mean(Beigvseries[idxeta,:]),np.squeeze(Reigvecseries[idxeta,:,NE:])*np.mean(Beigvseries[idxeta,:]),np.squeeze(Leigvecseries[idxeta,:,:NE]),np.squeeze(Leigvecseries[idxeta,:,NE:])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

# ## keep the same sign
# if meanmE[0]<0:
#     meanmE,meanmI,meannE,meannI=-meanmE,-meanmI,-meannE,-meannI
#     mEvec,mIvec,nEvec,nIvec=-mEvec,-mIvec,-nEvec,-nIvec
''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

ax.scatter(nIvec,mIvec,s=5.0,c=cm[1],alpha=alphaval)
ax.scatter(nEvec,mEvec,s=5.0,c=cm[0],alpha=alphaval)

# ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='gray',linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')
print(etaseries[idxeta])

1000
(2, 2)
(2, 2)
1.0
<ipython-input-88-bddb1fcc8446>:58: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-88-bddb1fcc8446>:65: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-88-bddb1fcc8446>:73: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-88-bddb1fcc8446>:80: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-88-bddb1fcc8446>:91: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-88-bddb1fcc8446>:92: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
/Applications/Anaconda/anacon

## INDIVIDUAL STATISTICS
* PRIDICTED V.S. NUMERICAL

In [257]:
'''
# <l',r'> reflects the direction of changing
# PLOTING THE TINY CHANGE, PREDICTED USING PERTURBATION THEOREM
ensemble averaging
'''

xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]
## reconstruct Am
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
deltavE,deltavI=np.zeros((neta,ntrial,2,NE),dtype=complex),np.zeros((neta,ntrial,2,NE),dtype=complex)
perturbE,perturbI=np.zeros((neta,ntrial,2,NE),dtype=complex),np.zeros((neta,ntrial,2,NE),dtype=complex)
ntrial = np.shape(Leigvecseries)[1]
for itrial in range(ntrial):
    ## choose one trial and the random matrix
    # itrial=6
    gY=np.squeeze(gYbasexee[itrial,:,:])
    ## reconstruct the non-symmetric matrix (original random matrix with \eta=0)
    X0=gY.copy()
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ''' calculate s0'''
    Z = np.zeros((NE*2-1,NE*2-1))
    for i in range(NE*2-1):
        b1=eigvJ0[i+1]
        Z[i,i]=1/(b1-eigvJ0[0])
    S0=reig0[:,1:]@Z@leig0[:,1:].T
    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in x is positive, if negative then all x and y should change sign
    if (np.mean(tildex[:NE])*xAm[0,0])<0:
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))

    '''
    rho_prime=eta/(2*sqrt(1-eta**2))
    rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
    rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
    '''
    dh=etaseries[1]-etaseries[0]
    rhoseries=(1-np.sqrt(1-etaseries**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
    sqrt_1meta2= np.sqrt(1-etaseries**2)
    coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
    # EI,LR,:
    for i in range(neta-1):
        Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T-np.sqrt(coeffsqrtY[i])*gY
        leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
        rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E

        ## perturbed vector
        perturbE[i,itrial,0,:],perturbE[i,itrial,1,:]=leftperturb[:NE],rightperturb[:NE]
        perturbI[i,itrial,0,:],perturbI[i,itrial,1,:]=leftperturb[NE:],rightperturb[NE:]
        ## delta vector
        deltavE[i,itrial,0,:]=np.squeeze(Leigvecseries[i+1,itrial,:NE])-np.squeeze(Leigvecseries[i,itrial,:NE])
        deltavE[i,itrial,1,:]=np.squeeze(Reigvecseries[i+1,itrial,:NE])-np.squeeze(Reigvecseries[i,itrial,:NE])
        deltavI[i,itrial,0,:]=np.squeeze(Leigvecseries[i+1,itrial,NE:])-np.squeeze(Leigvecseries[i,itrial,NE:])
        deltavI[i,itrial,1,:]=np.squeeze(Reigvecseries[i+1,itrial,NE:])-np.squeeze(Reigvecseries[i,itrial,NE:])

test normal: (0.9999999999999842+3.497719858884776e-14j)
<ipython-input-257-36639bcd1e59>:49: ComplexWarning: Casting complex values to real discards the imaginary part
  Z[i,i]=1/(b1-eigvJ0[0])
test normal 2: (1.000000000000002-4.9698237028923744e-29j)
test normal 3: (-2.463522748698468-8.616712440937174e-14j)
eigenvalues: [-2.46352275+0.j]
<ipython-input-257-36639bcd1e59>:75: RuntimeWarning: divide by zero encountered in true_divide
  coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
test normal: (0.9999999999999454+6.845228397977299e-15j)
test normal 2: (1.0000000000000007-9.150786500563737e-29j)
test normal 3: (-2.5201921979971056-1.7251291202091576e-14j)
eigenvalues: [-2.5201922+0.j]
test normal: (1.0000000000000215-9.299296967025115e-16j)
test normal 2: (1.0000000000000002-1.1044052673094165e-29j)
test normal 3: (-2.5209971715060284+2.3443501350863956e-15j)
eigenvalues: [-2.52099717+0.j]
test normal: (0.9999999999999784+2.07946451853743e-1

In [258]:
xticks = np.linspace(-0.15,0.15,2)
xlims = [-0.15,0.15]
yticks = np.linspace(-0.15,0.15,2)
ylims = [-0.15,0.15]
import random
'''
# CHOOSE ONE TRIAL
'''
shiftepsilon=np.array([3/(1e2),3/(1e2)])
idxtrial,idxtrial_=8,3
figC,axC=plt.subplots(2,2,figsize=(4,4))
idxsamples=np.array([1,7])#[1,5,9])
dh=etaseries[1]-etaseries[0]
for i in range(len(idxsamples)):
    pEensembleL,pIensembleL=np.squeeze(perturbE[idxsamples[i],:,0,:]).flatten(),np.squeeze(perturbI[idxsamples[i],:,0,:]).flatten()
    pEensembleR,pIensembleR=np.squeeze(perturbE[idxsamples[i],:,1,:]).flatten(),np.squeeze(perturbI[idxsamples[i],:,1,:]).flatten()
    vEensembleL,vIensembleL=np.squeeze(deltavE[idxsamples[i],:,0,:]).flatten(),np.squeeze(deltavI[idxsamples[i],:,0,:]).flatten()
    vEensembleR,vIensembleR=np.squeeze(deltavE[idxsamples[i],:,1,:]).flatten(),np.squeeze(deltavI[idxsamples[i],:,1,:]).flatten()

    axC[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    axC[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    axC[0,i].scatter(pEensembleL,vEensembleL/dh,s=5,c='black',alpha=0.25)
    axC[0,i].scatter(pIensembleL+shiftepsilon[i],vIensembleL/dh-shiftepsilon[i],s=5,c='gray',alpha=0.5)
    axC[1,i].scatter(pEensembleR,vEensembleR/dh,s=5,c='black',alpha=0.25)
    axC[1,i].scatter(pIensembleR+shiftepsilon[i],vIensembleR/dh-shiftepsilon[i],s=5,c='gray',alpha=0.5)
    print(shiftepsilon[i])
    axC[0,i].set_xticks([])
    axC[0,i].set_yticks([])
    axC[1,i].set_xticks([])
    axC[1,i].set_yticks([])

for i in range(len(idxsamples)):
    axC[0,i].set_xlim(xlims)
    axC[0,i].set_ylim(ylims)
    axC[1,i].set_xlim(xlims)
    axC[1,i].set_ylim(ylims)
axC[0,0].set_yticks(yticks)
axC[1,0].set_yticks(yticks)


'''
# CHOOSE TWO TRIALS
'''
shiftepsilon=np.array([2/(1e2),2/(1e2)])
idxtrial,idxtrial_=8,3
figM,axM=plt.subplots(2,2,figsize=(4,4))
idxsamples=np.array([1,7])#[1,5,9])
dh=etaseries[1]-etaseries[0]
for i in range(len(idxsamples)):
    pEensembleL,pIensembleL=np.squeeze(perturbE[idxsamples[i],:,0,:]).flatten(),np.squeeze(perturbI[idxsamples[i],:,0,:]).flatten()
    pEensembleR,pIensembleR=np.squeeze(perturbE[idxsamples[i],:,1,:]).flatten(),np.squeeze(perturbI[idxsamples[i],:,1,:]).flatten()
    vEensembleL,vIensembleL=np.squeeze(deltavE[idxsamples[i],:,0,:]).flatten(),np.squeeze(deltavI[idxsamples[i],:,0,:]).flatten()
    vEensembleR,vIensembleR=np.squeeze(deltavE[idxsamples[i],:,1,:]).flatten(),np.squeeze(deltavI[idxsamples[i],:,1,:]).flatten()
    print(vEensembleR[:5])
    random.shuffle(np.squeeze(vEensembleL))
    random.shuffle(np.squeeze(vEensembleR))
    random.shuffle(np.squeeze(vIensembleL))
    random.shuffle(np.squeeze(vIensembleR))

    # vEensembleL,vIensembleL,vEensembleR,vIensembleR=random.shuffle(vEensembleL.copy()),random.shuffle(vIensembleL.copy()),random.shuffle(vEensembleR.copy()),random.shuffle(vIensembleR.copy())
    print(vEensembleL[:5])
    axM[0,i].plot(xticks,yticks,color='darkred',linestyle='--')
    axM[1,i].plot(xticks,yticks,color='darkred',linestyle='--')
    axM[0,i].scatter(pEensembleL,vEensembleL/dh,s=5,c='black',alpha=0.25)
    axM[0,i].scatter(pIensembleL+shiftepsilon[i],vIensembleL/dh-shiftepsilon[i],s=5,c='gray',alpha=0.5)
    axM[1,i].scatter(pEensembleR,vEensembleR/dh,s=5,c='black',alpha=0.25)
    axM[1,i].scatter(pIensembleR+shiftepsilon[i],vIensembleR/dh-shiftepsilon[i],s=5,c='gray',alpha=0.5)
    print(shiftepsilon[i])
    axM[0,i].set_xticks([])
    axM[0,i].set_yticks([])
    axM[1,i].set_xticks([])
    axM[1,i].set_yticks([])

for i in range(len(idxsamples)):
    axM[0,i].set_xlim(xlims)
    axM[0,i].set_ylim(ylims)
    axM[1,i].set_xlim(xlims)
    axM[1,i].set_ylim(ylims)
axM[0,0].set_yticks(yticks)
axM[1,0].set_yticks(yticks)

0.03
0.03
[ 5.55748141e-04+4.92302920e-18j -1.61163384e-04+4.99534072e-18j
  7.20756767e-05+5.98794394e-18j -1.36229812e-04+5.47327689e-18j
  5.64835273e-05+4.99819528e-18j]
[-0.00112181-3.36968182e-14j -0.00033841-5.69093050e-14j
  0.00011455+7.33747288e-14j -0.00067715+1.44936392e-14j
  0.0004233 -1.32042896e-14j]
0.02
[ 1.05336554e-03-4.06174091e-17j -3.88572913e-04-4.77688757e-17j
 -1.15889645e-04-5.43581925e-17j -5.70092571e-04-5.13807861e-17j
  2.52438671e-05-4.57977419e-17j]
[ 0.00089086-4.37897275e-14j -0.00151341-3.69549162e-14j
 -0.00024394-2.73573613e-14j -0.00084569-8.03927684e-15j
 -0.00117842-1.95598926e-14j]
0.02
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:138: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order, subok=True)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:138: ComplexWarning: Casting complex values to real di

## Convert

In [14]:
## compare convert
strfile='2021-03-31-12_40_37_perturbationHomoetaComp_data'

## convert
strfile='2021-03-28-23_26_33_perturbationConvert_data'
data = np.load(strfile+'.npz')
eta=data['eta']
neta=len(etaseries)
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)
print('gain average:',gaverage)


scale: 1000.0
JEE: 1.0
eta: [1. 1. 1.] ; signs: [1. 1. 1.]
gain average: 1.0


In [33]:
'''
# <l',r'> reflects the direction of changing
'''

'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]
## reconstruct Am
nrank=1
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
vcovEmat,vcovImat=np.zeros((ntrial,2,2)),np.zeros((ntrial,2,2))
vcov0Emat,vcov0Imat=np.zeros((ntrial,2,2)),np.zeros((ntrial,2,2))
pcovEmat,pcovImat=np.zeros((ntrial,2,2)),np.zeros((ntrial,2,2))
directionvec0E,directionvec0I,directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
ampvec0E,ampvec0I,ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros(2),np.zeros(2),np.zeros(2),np.zeros(2),np.zeros(2),np.zeros(2)
overlapperturb,overlapvector = np.zeros((2,2,2,NE),dtype=complex),np.zeros((2,2,2,NE),dtype=complex)
ntrial = np.shape(Leigvecseries)[1]
for itrial in range(ntrial):
    ## choose one trial and the random matrix
    gY=np.squeeze(gYbasexee[itrial,:,:])
    # # NORMAL
    # x0,y0=np.reshape(Reigvecseries[0,itrial,:],(2*NE,1)),np.reshape(Leigvecseries[0,itrial,:],(2*NE,1))
    # COMPARE CONVERT
    x0,y0=np.reshape(Reigvecseries[0,itrial,:],(2*NE,1)),np.reshape(Leigvecseries[0,itrial,:],(2*NE,1)) # 0 is after calculation
    i = 1
    overlapvector[i,0,0,:]=np.squeeze(Leigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Leigvecseries[i,itrial,:NE]))
    overlapvector[i,1,0,:]=np.squeeze(Leigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Leigvecseries[i,itrial,NE:]))
    overlapvector[i,0,1,:]=np.squeeze(Reigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Reigvecseries[i,itrial,:NE]))
    overlapvector[i,1,1,:]=np.squeeze(Reigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Reigvecseries[i,itrial,NE:]))
    dh=1#etaseries[1]-etaseries[0]
    rhoseries=(1-np.sqrt(1-eta**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
    sqrt_1meta2= np.sqrt(1-eta**2)
    coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
    # EI,LR,:
    Jprimemat=np.sqrt(coeffsqrtYT)*gY.copy().T-np.sqrt(coeffsqrtY)*gY
    leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
    rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E
    i = 1
    overlapperturb[i,0,0,:],overlapperturb[i,0,1,:]=leftperturb[:NE],rightperturb[:NE]
    overlapperturb[i,1,0,:],overlapperturb[i,1,1,:]=leftperturb[NE:],rightperturb[NE:]
    ## E population 
    # vectorcovE,perturbationcovE = np.zeros((neta,2,2)),np.zeros((neta,2,2))

    vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
    perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
    vcovEmat[itrial,:,:],vcovImat[itrial,:,:]=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
    pcovEmat[itrial,:,:],pcovImat[itrial,:,:]=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

    i = 0
    overlapvector[i,0,0,:]=np.squeeze(Leigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Leigvecseries[i,itrial,:NE]))
    overlapvector[i,1,0,:]=np.squeeze(Leigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Leigvecseries[i,itrial,NE:]))
    overlapvector[i,0,1,:]=np.squeeze(Reigvecseries[i,itrial,:NE])-np.mean(np.squeeze(Reigvecseries[i,itrial,:NE]))
    overlapvector[i,1,1,:]=np.squeeze(Reigvecseries[i,itrial,NE:])-np.mean(np.squeeze(Reigvecseries[i,itrial,NE:]))

    vector0Emat,vector0Imat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
    vcov0Emat[itrial,:,:],vcov0Imat[itrial,:,:]=vector0Emat@vector0Emat.T,vector0Imat@vector0Imat.T

## averaging
vcovEmatavg,vcovImatavg=np.mean(vcovEmat,axis=0),np.mean(vcovImat,axis=0)
pcovEmatavg,pcovImatavg=np.mean(pcovEmat,axis=0),np.mean(pcovImat,axis=0)
vvE,dirvE=la.eig(np.squeeze(vcovEmatavg))
vvI,dirvI=la.eig(np.squeeze(vcovImatavg))
vpE,dirpE=la.eig(np.squeeze(pcovEmatavg))
vpI,dirpI=la.eig(np.squeeze(pcovImatavg))
directionvecE,directionvecI=dirvE,dirvI
directionperturbE,directionperturbI=dirpE,dirpI
ampvecE,ampvecI=vvE,vvI
ampperturbE,ampperturbI=vpE,vpI

vcov0Ematavg,vcov0Imatavg=np.mean(vcov0Emat,axis=0),np.mean(vcov0Imat,axis=0)
vvE,dirvE=la.eig(np.squeeze(vcov0Ematavg))
vvI,dirvI=la.eig(np.squeeze(vcov0Imatavg))
directionvec0E,directionvec0I=dirvE,dirvI
ampvec0E,ampvec0I=vvE,vvI



<ipython-input-33-ec5bf93ce794>:55: ComplexWarning: Casting complex values to real discards the imaginary part
  vcovEmat[itrial,:,:],vcovImat[itrial,:,:]=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
<ipython-input-33-ec5bf93ce794>:56: ComplexWarning: Casting complex values to real discards the imaginary part
  pcovEmat[itrial,:,:],pcovImat[itrial,:,:]=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T
<ipython-input-33-ec5bf93ce794>:65: ComplexWarning: Casting complex values to real discards the imaginary part
  vcov0Emat[itrial,:,:],vcov0Imat[itrial,:,:]=vector0Emat@vector0Emat.T,vector0Imat@vector0Imat.T


In [34]:
# ## averaging
# directionvecEavg,directionvecIavg,directionperturbEavg,directionperturbIavg=np.mean(directionvecE,axis=1),np.mean(directionvecI,axis=1),np.mean(directionperturbE,axis=1),np.mean(directionperturbI,axis=1)
# ampvecEavg,ampvecIavg,ampperturbEavg,ampperturbIavg=np.mean(ampvecE,axis=1),np.mean(ampvecI,axis=1),np.mean(ampperturbE,axis=1),np.mean(ampperturbI,axis=1)

fig,ax=plt.subplots(figsize=(5,5)) 
idxeta=1
heta=1
ax.plot([0,0+ampvecE[0]*directionvecE[0,0]],[0,0+ampvecE[0]*directionvecE[1,0]],color='r',linestyle='-',linewidth=1.5)
ax.plot([0,0+ampvecI[0]*directionvecI[0,0]],[0,0+ampvecI[0]*directionvecI[1,0]],color='orange',linestyle='-',linewidth=1.5)
# ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
# ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)
vecEx,vecEy=ampvec0E[0]*directionvec0E[0,0],ampvec0E[0]*directionvec0E[1,0]
vecIx,vecIy=ampvec0I[0]*directionvec0I[0,0],ampvec0I[0]*directionvec0I[1,0]
ax.plot([vecEx,vecEx+ampperturbE[1]*directionperturbE[0,1]*heta],[vecEy,vecEy+ampperturbE[1]*directionperturbE[1,1]*heta],color='b',linestyle='--',linewidth=1.5)
ax.plot([vecIx,vecIx+ampperturbI[1]*directionperturbI[0,1]*heta],[vecIy,vecIy+ampperturbI[1]*directionperturbI[1,1]*heta],color='g',linestyle='--',linewidth=1.5)
    # ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,0]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,0]*heta],color='b',linestyle='--',linewidth=1.5)
    # ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,0]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,0]*heta],color='g',linestyle='--',linewidth=1.5)

# xticks = np.linspace(-0.15,0.15,2)
# xlims = [-0.15,0.15]
# yticks = np.linspace(-0.15,0.15,2)
# ylims = [-0.15,0.15]
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
ax.set_aspect('equal')

## noise covariance between left and right eigenvectors

In [28]:
strfile = '2021-04-01-12_34_52_perturbationHomoeta_data'
strfile = '2021-04-01-14_58_21_perturbationHomoeta_data'
data = np.load(strfile+'.npz')
etaseries=data['etaseries']
neta=len(etaseries)
etaset=data['etaset']
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)
print('gain average:',gaverage)

## reconstruct Am
nrank=1
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
DeltaL_theo,DeltaR_theo,DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_math,DeltaLR_theo,DeltaLR_num = np.zeros(neta,dtype=complex),np.zeros((neta,ntrial),dtype=complex),np.zeros((neta,ntrial),dtype=complex)
for idxeta in range(neta):
    ## choose one specific degree of symmetry
    etaloop = etaseries[idxeta]
    ## all vector variables (numerical)
    rperturbvec, lperturbvec = np.zeros((ntrial,NE*2)),np.zeros((ntrial,NE*2))
    ## J_0, the first set of eigenvectors or we use Am.copy()+Xprime
    asqr=(1-np.sqrt(1-etaloop**2))/2.0 
    aampT = np.sqrt(asqr) # for rho_T
    aampO = np.sqrt(1-aampT**2) # for rho
    # Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
    Jinit,Jeta = np.zeros((ntrial,NE*2,NE*2)),np.zeros((ntrial,NE*2,NE*2))
    Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2)),np.zeros((ntrial,NE*2))
    for itrial in range(ntrial):
        Xsym = np.squeeze(gYbasexee[itrial,:,:])
        XsymT = Xsym.copy().T
        Jinit[itrial,:,:] = Am.copy()+Xsym.copy()
        Jeta[itrial,:,:]  = Am.copy()+aampT*XsymT.copy()+aampO*Xsym.copy()
        # initial eigenvectors
        Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[idxeta,itrial,:]).copy(),np.squeeze(Reigvecseries[idxeta,itrial,:]).copy()

    # DeltaL/R_theo two types, one version before simplification and the other after analytical simplification
    for itrial in range(ntrial):
        l0,r0=np.reshape(Leigvecinit[itrial,:],(NE*2,1)),np.reshape(Reigvecinit[itrial,:],(NE*2,1))
        Jetag,J0=np.squeeze(Jeta[itrial,:,:]),np.squeeze(Jinit[itrial,:,:])
        # theoretical value
        changeL = np.reshape(l0.T@(Jetag-J0)/eigvAm[0],(NE*2,1))
        changeR = np.reshape((Jetag-J0)@r0/eigvAm[0],(NE*2,1))
        DeltaL_theo[idxeta,itrial,:],DeltaR_theo[idxeta,itrial,:]=np.squeeze(changeL),np.squeeze(changeR)
        DeltaLR_theo[idxeta,itrial]=np.squeeze(changeL.T@changeR)
        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:]),(NE*2,1))- np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:]),(NE*2,1))-np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        DeltaL_num[idxeta,itrial,:],DeltaR_num[idxeta,itrial,:]=np.squeeze(changeR_num),np.squeeze(changeL_num)
        DeltaLR_num[idxeta,itrial]=np.squeeze(changeL_num.T@changeR_num)
    DeltaLR_math[idxeta]=(2*aampT*aampO*gaverage**2-2*aampT*gaverage**2)/eigvAm[0]**2
    
fig,ax=plt.subplots(figsize=(5,5))
ax.plot(DeltaLR_math,np.mean(DeltaLR_num,axis=1))
ax.set_aspect('equal')


scale: 500.0
JEE: 0.5
eta: [1. 1. 1.] ; signs: [1. 1. 1.]
gain average: 0.8
<ipython-input-28-0dccb9d88bf9>:68: ComplexWarning: Casting complex values to real discards the imaginary part
  Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[idxeta,itrial,:]).copy(),np.squeeze(Reigvecseries[idxeta,itrial,:]).copy()


## heterogeneous symmetry


In [13]:
strfile = '2021-04-01-12_34_52_perturbationHomoeta_data'
# strfile = '2021-04-01-14_58_21_perturbationHomoeta_data'
# strfile='2021-04-01-18_24_33_perturbationHetereta_data'

# strfile='2021-03-27-22_40_39_perturbationHomoeta_data' # 750 0.8
strfile='2021-03-27-23_15_34_perturbationHomoeta_data' # 500 0.8
# strfile='2021-03-28-16_34_16_perturbationHomoeta_data' #1000 0.2

# strfile='2021-03-28-10_09_24_perturbationHomoeta_data' # 750 0.2
# strfile='2021-03-28-10_14_57_perturbationHomoeta_data' # 500 0.2
# strfile='2021-03-28-16_15_47_perturbationHomoeta_data' # 1000 0.8

## reverse JE and JI
# strfile='2021-03-30-16_38_37_perturbationHomoeta_data' # 1000
# strfile='2021-03-30-16_55_54_perturbationHomoeta_data' # 750

## JI 4.5 JE 0.5
# strfile='2021-03-30-17_04_25_perturbationHomoeta_data' # 750
# strfile='2021-03-30-17_47_12_perturbationHomoeta_data' # 1000
data = np.load(strfile+'.npz')
etaseries=data['etaseries']
neta=len(etaseries)
etaset=data['etaset']
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
x0,y0=data['x0'],data['y0']
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)
print('gain average:',gaverage)

## reconstruct Am
nrank=1
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
DeltaL_theo,DeltaR_theo,DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_math,DeltaLR_theo,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial),dtype=complex),np.zeros((neta,ntrial),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2))
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2)),np.zeros((ntrial,NE*2))
## initial vectors LEFT & RIGHT same for identical trial

for itrial in range(ntrial):
    Xsym = np.squeeze(gYbasexee[itrial,:,:])
    XsymT = Xsym.copy().T
    Jinit[itrial,:,:] = Am.copy()+Xsym.copy()
    # initial eigenvectors
    Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[0,itrial,:]).copy(),np.squeeze(Reigvecseries[0,itrial,:]).copy()

    Jeta = np.zeros((neta,NE*2,NE*2)) # refreshing for each random realization
    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        etaloop = etaseries[idxeta]*np.ones(3)
        # print('eta:',etaloop)
        etaloop=etaloop.copy()*coeffeta
        # print('--eta:',etaloop)
        Xinit = Xsym.copy()
        ## J_0, the first set of eigenvectors or we use Am.copy()+Xprime
        asqr=(1-np.sqrt(1-etaloop[0]**2))/2.0 
        rhoE=asqr
        aampT = np.sqrt(asqr) # for rho_T
        aampO = np.sqrt(1-aampT**2) # for rho
        Xinit[:NE,:NE]=aampT*XsymT[:NE,:NE].copy()+aampO*Xsym[:NE,:NE].copy()

        asqr=(1-np.sqrt(1-etaloop[1]**2))/2.0 
        rhoB=asqr
        aampT = np.sqrt(asqr) # for rho_T
        aampO = np.sqrt(1-aampT**2) # for rho
        Xinit[:NE,NE:]=aampT*XsymT[:NE,NE:].copy()+aampO*Xsym[:NE,NE:].copy()
        Xinit[NE:,:NE]=aampT*XsymT[NE:,:NE].copy()+aampO*Xsym[NE:,:NE].copy()

        asqr=(1-np.sqrt(1-etaloop[2]**2))/2.0 
        rhoI=asqr
        aampT = np.sqrt(asqr) # for rho_T
        aampO = np.sqrt(1-aampT**2) # for rho
        Xinit[NE:,NE:]=aampT*XsymT[NE:,NE:].copy()+aampO*Xsym[NE:,NE:].copy()

        Jeta[idxeta,:,:]  = Am.copy()+Xinit.copy()

        ### >>>>>>>>>>>>  extracting all useful variables >>>>>>>>>
        Jetag,J0=np.squeeze(Jeta[idxeta,:,:]),np.squeeze(Jinit[itrial,:,:])
        # theoretical value
        l0,r0=np.reshape(Leigvecinit[itrial,:],(2*NE,1)),np.reshape(Reigvecinit[itrial,:],(2*NE,1))
        changeL = np.reshape(l0.T@(Jetag-J0)/eigvAm[0],(NE*2,1))
        changeR = np.reshape((Jetag-J0)@r0/eigvAm[0],(NE*2,1))
        DeltaL_theo[idxeta,itrial,:],DeltaR_theo[idxeta,itrial,:]=np.squeeze(changeL),np.squeeze(changeR)
        DeltaLR_theo[idxeta,itrial]=np.squeeze(changeL.T@changeR)
        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:]),(NE*2,1))-l0#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:]),(NE*2,1))-r0#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        DeltaL_num[idxeta,itrial,:],DeltaR_num[idxeta,itrial,:]=np.squeeze(changeR_num),np.squeeze(changeL_num)
        DeltaLR_num[idxeta,itrial]=np.squeeze(changeL_num.T@changeR_num)
        # Excitatory
        DeltaLR_math[idxeta,0]=(np.sqrt((1+rhoE)*rhoE)+np.sqrt((1+rhoB)*rhoB)-np.sqrt(rhoE)-np.sqrt(rhoB))*JEE/(JEE-JEI)*gaverage**2/eigvAm[0]**2

        DeltaLR_math[idxeta,1]=(np.sqrt((1+rhoI)*rhoI)+np.sqrt((1+rhoB)*rhoB)-np.sqrt(rhoI)-np.sqrt(rhoB))*(-JEI)/(JEE-JEI)*gaverage**2/eigvAm[0]**2        #(2*aampT*aampO*gaverage**2-2*aampT*gaverage**2)/eigvAm[0]**2
        DeltaLR_math[idxeta,2]=-DeltaLR_math[idxeta,0]-DeltaLR_math[idxeta,1]




    






FileNotFoundError: [Errno 2] No such file or directory: '2021-03-27-23_15_34_perturbationHomoeta_data.npz'

In [12]:
fig,ax=plt.subplots(figsize=(5,5))
ax.plot(DeltaLR_math[:,2],np.mean(np.squeeze(DeltaLR_num[:,:]),axis=1),color='r',linewidth=1.5,label='num v.s. theo')
ax.plot(np.mean(DeltaLR_theo,axis=1),np.mean(DeltaLR_num,axis=1),color='b',linewidth=1.5,label='num v.s. simp')
ax.set_aspect('equal')

/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


## sparse symmetric connectivity

In [125]:
# strfile='2021-04-06-16_05_35_perturbationPureSubEEHetereta_data'
# strfile='2021-04-06-15_41_29_perturbationPureEEHetereta_data'
# strfile='2021-04-06-13_22_26_perturbationHetereta_data' #eta: [1.  0.6 0.3] ; signs: [1. 1. 1.] gain average: 1.0 ppercent [1. 1. 1.] coeffeta: [1.  0.6 0.3]
strfile ='2021-04-06-13_09_06_perturbationSparseHetereta_data' #scale: 750.0 JEE: 3.5 eta: [1.  0.6 0.3] ; signs: [1. 1. 1.]gain average: 1.0 ppercent [0.8 0.6 0.4] coeffeta: [1.  0.6 0.3]
data = np.load(strfile+'.npz')
etaseries=data['etaseries']
neta=len(etaseries)
etaset=data['etaset']
eigvAm=data['eigvAm']
eigvalseries=data['eigvalseries']
statesReig=data['statesReig']
statesLeig=data['statesLeig']
meigvecAm=data['meigvecAm']
neigvecAm=data['neigvecAm']
gYbasexee=data['gYbasexee']
Jparameters=data['Jparameters']
ppercent = data['ppercent']
# right and left eigenvectors
Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
ntrial=np.shape(Reigvecseries)[1]
xAm,yAm=data['xAm'],data['yAm']
Randgain=data['Randgain']
xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
coeffeta=data['coeffeta']

print('scale:',Jparameters[4])
NE=int(Jparameters[4])
N = NE*2
print('JEE:',Jparameters[0])
signeta=data['signeta']
gaverage=data['gaverage']
JE,JI=Jparameters[0],Jparameters[1]
print('eta:',etaset,'; signs:',signeta)
print('gain average:',gaverage)
print('ppercent',ppercent)
print('coeffeta:',coeffeta)
## reconstruct Am
nrank=1
JEE,JEI,JIE,JII=Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]
NI=NE
Nparams=np.array([NE,NI])
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)

scale: 750.0
JEE: 3.5
eta: [1.  0.6 0.3] ; signs: [1. 1. 1.]
gain average: 1.0
ppercent [0.8 0.6 0.4]
coeffeta: [1.  0.6 0.3]


In [114]:
'''
check the structure of XX and compare it with the theoretcial image
'''
Xsym = np.squeeze(gYbasexee[neta-1,:,:])
XsymT = Xsym.copy().T
## choose one specific degree of symmetry
etaset = etaseries[neta-1]*np.ones(3)
# print('eta:',etaloop)
etaset=etaset.copy()*coeffeta
# print('--eta:',etaloop)
Xinit = Xsym.copy()
### >>>>>>>>. half Exc sym >>>>>>>>>.
# modified
hNE=int(ppercent[0]*NE)
hrelist = np.random.permutation(NE)
xh = np.squeeze(hrelist[:hNE])
[xeehNE,yeehNE]=np.meshgrid(xh,xh)
asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
aamp=np.sqrt(asqr)
Xinit[xeehNE,yeehNE]=signeta[0]*aamp*XsymT[xeehNE,yeehNE].copy()+np.sqrt(1-aamp**2)*Xsym[xeehNE,yeehNE].copy()
## EI IE CHOOSINGF
hNE=int(ppercent[1]*NE)
hrelist = np.random.permutation(NE)
xh = np.squeeze(hrelist[:hNE])
vrelist = np.random.permutation(NE)
yh = np.squeeze(vrelist[:hNE])
[xeihNE,yeihNE]=np.meshgrid(xh,yh)
xeihNE +=NE
## EI IE##
asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
aamp=np.sqrt(asqr)
Xinit[xeihNE,yeihNE]=signeta[0]*aamp*XsymT[xeihNE,yeihNE].copy()+np.sqrt(1-aamp**2)*Xsym[xeihNE,yeihNE].copy()
Xinit[yeihNE,xeihNE]=signeta[0]*aamp*XsymT[yeihNE,xeihNE].copy()+np.sqrt(1-aamp**2)*Xsym[yeihNE,xeihNE].copy()

## II CHOOSING
hNE=int(ppercent[2]*NE)
hrelist = np.random.permutation(NE)
xh = np.squeeze(hrelist[:hNE])
[xiihNE,yiihNE]=np.meshgrid(xh,xh)
xiihNE,yiihNE = xiihNE+NE,yiihNE+NE
## II ##
asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
aamp=np.sqrt(asqr)
Xinit[xiihNE,yiihNE]=signeta[0]*aamp*XsymT[xiihNE,yiihNE].copy()+np.sqrt(1-aamp**2)*Xsym[xiihNE,yiihNE].copy()

Xfinal_random = Xinit.copy()
kcut = 7
Xorder = np.zeros((kcut,NE*2,NE*2))
for i in range(kcut):
    Xorder[i,:,:] = la.matrix_power(Xfinal_random.copy(),i)
fig,ax=plt.subplots(2,3,figsize=(6,6))
BINS = 15
for i in range(2):
    for j in range(3):
        iorder = i*3+j
        ax[i][j].imshow(Xorder[iorder,NE-BINS:NE+BINS,NE-BINS:NE+BINS],cmap='gray',clim=([-.6,0.6]))

<ipython-input-114-86211b010653>:50: ComplexWarning: Casting complex values to real discards the imaginary part
  Xorder[i,:,:] = la.matrix_power(Xfinal_random.copy(),i)


In [115]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
xticks = np.linspace(-0.2,0.25,2)
xlims = [-0.2,0.25]
yticks = np.linspace(-0.2,0.25,2)
ylims = [-0.2,0.25]
xticks = np.linspace(-0.1,0.1,2)
xlims = [-0.1,0.1]
yticks = np.linspace(-0.1,0.1,2)
ylims = [-0.1,0.10]
# xticks = np.linspace(-0.15,0.10,2)
# xlims = [-0.15,0.10]
# yticks = np.linspace(-0.06,0.01,2)
# ylims = [-0.06,0.01]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=5#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)
print(NE)
'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE]),np.squeeze(Reigvecseries[idxeta,:,NE:]),np.squeeze(Leigvecseries[idxeta,:,:NE]),np.squeeze(Leigvecseries[idxeta,:,NE:])
mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
scale_std=3.0
for irank in range(nrank):
    meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere]=meanmEtotal
    meanmE[irank]=np.mean(mEvec)

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere]=meanmItotal
    meanmI[irank]=np.mean(mIvec)
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere]=meannEtotal
    meannE[irank]=np.mean(nEvec)

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere]=meannItotal
    meannI[irank]=np.mean(nIvec)

''' calculate directions '''
noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
for irank in range(nrank):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
# m1n1
covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
_,dirvecE=la.eig(covdirE)
_,dirvecI=la.eig(covdirI)  

# ax.scatter(nIvec,mIvec,s=5.0,c='b',alpha=alphaval)#cm[1],alpha=alphaval)
# ax.scatter(nEvec,mEvec,s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)

# # ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color='gray',linestyle='--',linewidth=1.5)
# # ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color='gray',linestyle='--',linewidth=1.5)
# ax.plot([meannE[0],meannE[0]+dirvecE[0,0]],[meanmE[0],meanmE[0]+dirvecE[1,0]],color=edgv,linestyle='--',linewidth=1.5)
# ax.plot([meannE[0],meannE[0]+dirvecE[0,1]],[meanmE[0],meanmE[0]+dirvecE[1,1]],color=edgv,linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+dirvecI[0,0]],[meanmI[0],meanmI[0]+dirvecI[1,0]],color=edgv,linestyle='--',linewidth=1.5)
# ax.plot([meannI[0],meannI[0]+dirvecI[0,1]],[meanmI[0],meanmI[0]+dirvecI[1,1]],color=edgv,linestyle='--',linewidth=1.5)

# confidence_ellipse(np.real(nEvec),np.real(mEvec),ax,edgecolor=edgv)
# confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor=edgv)

ax.plot([0,0+dirvecE[0,0]],[0,0+dirvecE[1,0]],color='r',linestyle='--',linewidth=1.5)
ax.plot([0,0+dirvecE[0,1]],[0,0+dirvecE[1,1]],color='r',linestyle='--',linewidth=1.5)
ax.plot([0,0+dirvecI[0,0]],[0,0+dirvecI[1,0]],color='b',linestyle='--',linewidth=1.5)
ax.plot([0,0+dirvecI[0,1]],[0,0+dirvecI[1,1]],color='b',linestyle='--',linewidth=1.5)

confidence_ellipse(np.real(nEvec)-meannE,np.real(mEvec)-meanmE,ax,edgecolor='r')
confidence_ellipse(np.real(nIvec)-meannI,np.real(mIvec)-meanmI,ax,edgecolor='b')


ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.set_aspect('equal')
print(etaseries[idxeta])

750
(2, 2)
(2, 2)
1.0
<ipython-input-115-717dc8b3c859>:42: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-115-717dc8b3c859>:49: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-115-717dc8b3c859>:57: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-115-717dc8b3c859>:64: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-115-717dc8b3c859>:71: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-115-717dc8b3c859>:72: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


In [66]:
'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
# xticks = np.linspace(-0.2,0.25,2)
# xlims = [-0.2,0.25]
# yticks = np.linspace(-0.2,0.25,2)
# ylims = [-0.2,0.25]
xticks = np.linspace(-0.15,0.10,2)
xlims = [-0.15,0.10]
yticks = np.linspace(-0.06,0.01,2)
ylims = [-0.06,0.01]
theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''

idxtrial=5#16#
idxeta=9#9# 
alphaval=0.10
edgv='black'
cm='br'
fig,ax=plt.subplots(figsize=(5,3))  
nrank=1
ntrial = np.shape(Reigvecseries)[1]
neta = np.shape(Reigvecseries)[0]
NE=int(NE)
scale_std=3.0
print(NE)
## >>>>>>> record theta >>>>>>>>
covthetaE,covthetaI = np.zeros((neta,2,2)),np.zeros((neta,2,2))
covlambdaE,covlambdaI = np.zeros((neta,2)),np.zeros((neta,2))
for idxeta in range(neta):
    '''loading vector changing'''
    meanmE,meanmI,meannE,meannI=np.zeros(nrank),np.zeros(nrank),np.zeros(nrank),np.zeros(nrank)

    mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE]),np.squeeze(Reigvecseries[idxeta,:,NE:]),np.squeeze(Leigvecseries[idxeta,:,:NE]),np.squeeze(Leigvecseries[idxeta,:,NE:])
    mEvec,mIvec,nEvec,nIvec=mEvec.flatten(),mIvec.flatten(),nEvec.flatten(),nIvec.flatten()
    for irank in range(nrank):
        meanmEtotal,stdmEtotal = np.mean(mEvec),np.std(mEvec)
        varmE = mEvec - meanmEtotal
        idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
        mEvec[idxwhere]=meanmEtotal
        meanmE[irank]=np.mean(mEvec)

        # puring I
        meanmItotal,stdmItotal = np.mean(mIvec),np.std(mIvec)
        varmI = mIvec - meanmItotal
        idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
        mIvec[idxwhere]=meanmItotal
        meanmI[irank]=np.mean(mIvec)
        
        # n vector
        # puring E
        meannEtotal,stdnEtotal = np.mean(nEvec),np.std(nEvec)
        varnE = nEvec - meannEtotal
        idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
        nEvec[idxwhere]=meannEtotal
        meannE[irank]=np.mean(nEvec)

        # puring I
        meannItotal,stdnItotal = np.mean(nIvec),np.std(nIvec)
        varnI = nIvec - meannItotal
        idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
        nIvec[idxwhere]=meannItotal
        meannI[irank]=np.mean(nIvec)

    ''' calculate directions '''
    noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
    dirvecE,dirvecI=np.zeros((2,2)),np.zeros((2,2))
    for irank in range(nrank):
        # E 0 M 1 N
        noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
        noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]
    # m1n1
    covdirE,covdirI=np.squeeze(noiseE[:,:,0]).T@np.squeeze(noiseE[:,:,0]),np.squeeze(noiseI[:,:,0]).T@np.squeeze(noiseI[:,:,0])
    lambdaE,dirvecE=la.eig(covdirE)
    lambdaI,dirvecI=la.eig(covdirI) 
    covthetaE[idxeta,:,:],covthetaI[idxeta,:,:]= dirvecE,dirvecI
    covlambdaE[idxeta,:],covlambdaI[idxeta,:]= lambdaE,lambdaI

750
<ipython-input-66-f3cb1455544f>:43: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec)
<ipython-input-66-f3cb1455544f>:50: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec)
<ipython-input-66-f3cb1455544f>:58: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec)
<ipython-input-66-f3cb1455544f>:65: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec)
<ipython-input-66-f3cb1455544f>:72: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec-meannE[irank],mEvec-meanmE[irank]
<ipython-input-66-f3cb1455544f>:73: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseI[:,0,irank],noiseI[:,1,irank]= nIvec-meannI[irank],mIvec-meanmI[irank]


In [89]:
thetavalE,thetavalI = np.zeros((neta,2)),np.zeros((neta,2))
angleE,angleI = np.zeros((neta,2)),np.zeros((neta,2))
for idxeta in range(neta):
    for idir in range(2):
        thetavalE[idxeta,idir]=covthetaE[idxeta,1,idir]/covthetaE[idxeta,0,idir]
        thetavalI[idxeta,idir]=covthetaI[idxeta,1,idir]/covthetaI[idxeta,0,idir]

        angleE[idxeta,idir]=np.arctan2(covthetaE[idxeta,1,idir],covthetaE[idxeta,0,idir])
        angleI[idxeta,idir]=np.arctan2(covthetaI[idxeta,1,idir],covthetaI[idxeta,0,idir])
cl='rbk'
fig,ax=plt.subplots(figsize=(5,5))
i=0
ax.plot(etaseries,angleE[:,i],color=cl[i],linewidth=1.5)
ax.plot(etaseries,angleI[:,i],color=cl[i],linewidth=1.5)
fig,ax=plt.subplots(figsize=(5,5))
i=0
ax.plot(etaseries[1:],thetavalE[1:,i],color=cl[i],linewidth=1.5)
ax.plot(etaseries[1:],thetavalI[1:,i],color=cl[i],linewidth=1.5)


[[-1.13458505 -0.8813795 ]
 [ 1.00211867  0.99788581]
 [ 1.27917683  0.78175275]
 [ 1.38130649  0.72395229]
 [ 1.42976236  0.69941693]
 [ 1.46384832  0.68313089]
 [ 1.48358696  0.67404205]
 [ 1.48488902  0.673451  ]
 [ 1.47749266  0.67682231]
 [ 1.45718217  0.686256  ]]


In [126]:
'''
Total Theoretical Solution for Covariance
'''

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
DeltaL_theo,DeltaR_theo,DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_math,DeltaLR_theo,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial),dtype=complex),np.zeros((neta,ntrial,3),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
## initial vectors LEFT & RIGHT same for identical trial

for itrial in range(ntrial):
    # initial eigenvectors
    Leigvecinit[itrial,:],Reigvecinit[itrial,:]=np.squeeze(Leigvecseries[0,itrial,:]).copy(),np.squeeze(Reigvecseries[0,itrial,:]).copy()
    l0,r0=np.reshape(Leigvecinit[itrial,:],(2*NE,1)),np.reshape(Reigvecinit[itrial,:],(2*NE,1))#np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        etaloop = etaseries[idxeta]*np.ones(3)
        # print('eta:',etaloop)
        etaloop=etaloop*coeffeta
        # etaloop=(etaloop*(ppercent**2)
        print('equivalent etaloop:',etaloop)

        asqr=(1-np.sqrt(1-etaloop[0]**2))/2.0 
        rhoE=asqr

        asqr=(1-np.sqrt(1-etaloop[1]**2))/2.0 
        rhoB=asqr

        asqr=(1-np.sqrt(1-etaloop[2]**2))/2.0 
        rhoI=asqr

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:]),(NE*2,1))-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:]),(NE*2,1))-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        DeltaL_num[idxeta,itrial,:],DeltaR_num[idxeta,itrial,:]=np.squeeze(changeL_num),np.squeeze(changeR_num)
        DeltaLR_num[idxeta,itrial,2]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])
   
        # EXCITATORY
        # JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE)    
        # -JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)    
        DeltaLR_math[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE))    
        DeltaLR_math[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)))
        # Inhibitory
        DeltaLR_math[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB))    
        DeltaLR_math[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[2]**2)*(np.sqrt((1-rhoI)*rhoI)-np.sqrt(rhoI)))

        DeltaLR_math[idxeta,2]=DeltaLR_math[idxeta,1]+DeltaLR_math[idxeta,0]

equivalent etaloop: [0. 0. 0.]
equivalent etaloop: [0.11111111 0.06666667 0.03333333]
equivalent etaloop: [0.22222222 0.13333333 0.06666667]
equivalent etaloop: [0.33333333 0.2        0.1       ]
equivalent etaloop: [0.44444444 0.26666667 0.13333333]
equivalent etaloop: [0.55555556 0.33333333 0.16666667]
equivalent etaloop: [0.66666667 0.4        0.2       ]
equivalent etaloop: [0.77777778 0.46666667 0.23333333]
equivalent etaloop: [0.88888889 0.53333333 0.26666667]
equivalent etaloop: [1.  0.6 0.3]
equivalent etaloop: [0. 0. 0.]
equivalent etaloop: [0.11111111 0.06666667 0.03333333]
equivalent etaloop: [0.22222222 0.13333333 0.06666667]
equivalent etaloop: [0.33333333 0.2        0.1       ]
equivalent etaloop: [0.44444444 0.26666667 0.13333333]
equivalent etaloop: [0.55555556 0.33333333 0.16666667]
equivalent etaloop: [0.66666667 0.4        0.2       ]
equivalent etaloop: [0.77777778 0.46666667 0.23333333]
equivalent etaloop: [0.88888889 0.53333333 0.26666667]
equivalent etaloop: [1. 

In [132]:
xticks = np.linspace(0.0,1.0,2)
xlims = [-0.1,1.1]
yticks = np.linspace(-0.035,0.00,2)
ylims = [-0.036,0.001]
# fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
# for i in range(3):
#     ax.plot(DeltaLR_math[:,i],np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,label='num v.s. theo')
# # ax.plot(np.mean(DeltaLR_theo,axis=1),np.mean(DeltaLR_num,axis=1),color='b',linewidth=1.5,label='num v.s. simp')
# ax.set_aspect('equal')

fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for i in range(2):
    ax.plot(etaseries,DeltaLR_math[:,i],color=cl[i],linewidth=1.5)
    # ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,linestyle='--')
# fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
for i in range(2,3):
    ax.plot(etaseries,DeltaLR_math[:,i],color=cl[i],linewidth=1.5)
    ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,i]),axis=1),color=cl[i],linewidth=1.5,linestyle='--')

ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)


/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
